In [2]:
import pandas as pd
import numpy as np
# import tensorflow as tf
# import os
# from tensorflow.keras import Sequential, Model, Input
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, concatenate
# from keras.layers.recurrent import LSTM
# from keras.layers import Dropout, Bidirectional
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.keras import Sequential, Model, Input
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, concatenate, BatchNormalization
# from keras.layers.recurrent import LSTM
# from keras.layers import Dropout, Bidirectional
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# import keras.backend as K
import sys
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
# from pytorch_tabnet.pretraining import TabNetPretrainer
from sklearn.model_selection import train_test_split
# import torch

In [3]:
# def addEpoch(date):
#     epoch = datetime.strptime(date, "%Y-%m-%d").timestamp()
#     return epoch

# def addUpIndicator(change_p):
#     if change_p >= 0.005:
#         return 'U'
#     else:
#         return 'D'
    
# def add3LIndicator(change_p):
#     if change_p >= 0.05:
#         return 'U'
#     elif change_p <= -0.05:
#         return 'D'
#     else:
#         return 'N'
    
# def addLabel(value):
#     if value == 'U':
#         a = 2
#     elif value == 'N':
#         a = 1
#     elif value == 'D':
#         a = 0
        
#     return a

# def convToNumpyArr(s, n):
#     str_arrays = s.split('], ')
#     #arr = np.empty(shape=[0, n])
#     arr = []
    
#     for array in str_arrays:
#         array = array.replace('[[', '')
#         array = array.replace('[', '')
#         array = array.replace(']', '')
#         array = array.replace(']]', '')
#         py_list = [float(s) for s in array.split(',')]
#         arr.append(py_list)
    
#     return arr

# from keras import backend as K

# def recall_m(y_true, y_pred):
#     y_true= tf.cast(y_true, dtype='float64')
#     y_pred= tf.cast(y_pred, dtype='float64')
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     y_true= tf.cast(y_true, dtype='float64')
#     y_pred= tf.cast(y_pred, dtype='float64')
    
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_score(y_true, y_pred):
#     y_true= tf.cast(y_true, dtype='float64')
#     y_pred= tf.cast(y_pred, dtype='float64')
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))



# def custom_loss(y_true, y_pred):
#     # calculating squared difference between target and predicted values 
#     y_true= tf.cast(y_true, dtype='float64')
#     y_pred= tf.cast(y_pred, dtype='float64')
#     loss = K.square(((y_pred - y_true)* [1.0, 3.0]))  # (batch_size, 2)
                
#     # summing both loss values along batch dimension 
#     loss = K.sum(loss, axis=1)        # (batch_size,)
    
#     return loss

def add3LIndicator_2D(change_p):
    if change_p >= 0.04:
        return 'V'
    elif change_p <= -0.04:
        return 'V'
    else:
        return 'NV'
    
def addLabel(value):
    if value == 'V':
        a = 1
    elif value == 'NV':
        a = 0
        
    return a

def add3LIndicator_3D(change_p):
    if change_p >= 0.04:
        return 'U'
    elif change_p <= -0.04:
        return 'D'
    else:
        return 'NV'
    
def addLabel_3D(value):
    if value == 'U':
        a = 1
    elif value == 'NV':
        a = 0
    else:
        a = 2
        
    return a


In [4]:
def label_(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (max_ >= y) :
        return 1
    elif (min_ <= -y):
        return 1
    else:
        return 0
    
def label_d(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (max_ >= y) :
        return 1
    elif (min_ <= -y):
        return 0
    else:
        return 2
    
def label_dd(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (min_ <= -y) :
        return 0
    elif (max_ >= y):
        return 1
    else:
        return 2
    
def calcEpsTTM(array):
    array = array.replace('[', '')
    array = array.replace(']', '')
    eps = [float(s) for s in array.split(', ')]
        
    return (eps[0]+eps[1]+eps[2]+eps[3])
    
def fundamental(symbol, tech_df):
    
    cal_df = pd.read_csv('fundamental/'+ symbol)
    cal_df = cal_df.set_index(['Unnamed: 0'])

    eps_list = []
    eps_act_list = []
    pre_eps_vec_list = []
    post_eps_vec_list = []

    for index, row in tech_df.iterrows():
        tech_df.loc[index, 'event'] = cal_df.loc[row['Date'], 'event']
        tech_df.loc[index, 'YoY'] = cal_df.loc[row['Date'], 'YoY']
        tech_df.loc[index, 'YoYRev'] = cal_df.loc[row['Date'], 'YoYRev']
        tech_df.loc[index, 'surprisePercent'] = cal_df.loc[row['Date'], 'surprisePercent']
        tech_df.loc[index, 'operatingMargin'] = cal_df.loc[row['Date'], 'operatingMargin']
        eps_list.append(cal_df.loc[row['Date'], 'eps_seq'])
        eps_act_list.append(cal_df.loc[row['Date'], 'eps_actual'])
        pre_eps_vec_list.append(cal_df.loc[row['Date'], 'pre_eps_vec'])
        post_eps_vec_list.append(cal_df.loc[row['Date'], 'post_eps_vec'])

    tech_df['eps_seq'] = eps_list
    tech_df['eps_actual'] = eps_act_list
    tech_df['pre_eps_vec'] = pre_eps_vec_list
    tech_df['post_eps_vec'] = post_eps_vec_list


    return tech_df

In [5]:
def calcEpsTTM(array):
    array = array.replace('[', '')
    array = array.replace(']', '')
    eps = [float(s) for s in array.split(', ')]
        
    return (eps[0]+eps[1]+eps[2]+eps[3])

def direction(x):
    if x >= 0.04:
        return 1
    elif x <= -0.04:
        return 0
    else:
        return 2
    
def vola(x):
    if x >= 0.03:
        return 1
    elif x <= -0.03:
        return 1
    else:
        return 0

In [1994]:
# t = pd.DataFrame({'tickers' :tickers_})
# t.to_csv('chosen_stocls_direction_v2_70.csv',index = False)

In [15]:
tickers = pd.read_csv('chosen_stocls_direction_v1.csv')['tickers'].unique().tolist()

In [2130]:
# pd.read_csv('chosen_stocls_direction_v2_70.csv')['tickers'].apply(lambda x: x.split('.')[0]).unique().tolist()

In [2131]:
# pd.read_csv('prod_up_tickers.csv')['tickers'].apply(lambda x: x.split('.')[0]).unique().tolist()

In [21]:
# import helper
import os
pd.options.mode.chained_assignment = None

# tick = []

# # av = ['OGN.csv', 'BF.csv', 'BRK.csv', 'CARR.csv', 'OTIS.csv','BRK-A.csv','MRNA.csv','TSLA.csv','NVDA.csv']
# av = ['OGN.csv', '.DS_Store', 'BF.csv', 'BRK.csv',['OGN.csv',
#  'NABL.csv',
#  '.DS_Store',
#  'VSCO.csv',
#  'GXO.csv',
#  'VMEO.csv',
#  'DTM.csv']]
# li = []

# for i in os.listdir('/Volumes/AARTHI/sematic_data/training_data'):
# #     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
#     if (i != '.ipynb_checkpoints') and (i not in av) :
#         li.append(i)
# # # # av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
# for i in os.listdir('fundamental'):
# #     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
#     if (i != '.ipynb_checkpoints') and (i not in av) :
#         tick.append(i)

# df_rate = pd.read_csv('Fed_rate.csv')
# spx = pd.read_csv('spx.csv')
# spx['Date'] = spx['Date'].astype(str)
# spx = spx[['Date','Adjusted_close']]
# spx.columns = ['Date','SNP_Close']

# df_list = []
# avoid = []
# for i in li:
#     if (i in tick) :

#         aa = i.split('.')[0]
#     #     if aa in al:

#         temp = pd.read_csv('/Volumes/AARTHI/sematic_data/training_data/' + i)
#         temp['ticker'] = i
#         # df_list.append(temp)

#         # temp = temp[(temp['Date'] >= '2014-01-01')].reset_index(drop = True)
#         try:
#             temp = helper.fundamental(aa, temp)
#             if (temp['YoYRev'].isnull().sum() == 0) & (temp['operatingMargin'].isnull().sum() == 0):
#                 temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#                 temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]

#                 fund_df = temp.merge(df_rate, on= ['Date']).drop(columns = ['Unnamed: 0'])
#                 fund_df = fund_df.merge(spx, on = 'Date')

#                 #Start with fundamental
#                 fund_df['Price_OED'] = None
#                 fund_df['SNP_Price_OED'] = None

#                 price_oed = None #Adjusted closing price on Earnings Day
#                 snp_price_oed = None #Adjusted closing price of S&P500 on Earnings Day
#                 prev_eps_seq = fund_df.iloc[0].eps_seq

#                 for i in range(fund_df.shape[0]):
#                     if (fund_df.iloc[i].event == 1.0):
#                         price_oed = fund_df.iloc[i].Adjusted_close
#                         snp_price_oed = fund_df.iloc[i].SNP_Close
#                         prev_eps_seq = fund_df.iloc[i].eps_seq
#                     elif (fund_df.iloc[i].eps_seq != prev_eps_seq):
#                         price_oed = fund_df.iloc[i].Adjusted_close
#                         snp_price_oed = fund_df.iloc[i].SNP_Close
#                         prev_eps_seq = fund_df.iloc[i].eps_seq
                        
#                     fund_df.iloc[i, fund_df.columns.get_loc('Price_OED')] = price_oed
#                     fund_df.iloc[i, fund_df.columns.get_loc('SNP_Price_OED')] = snp_price_oed

#                 fund_df["EpsTTM"] = fund_df["eps_actual"].apply(calcEpsTTM)
#                 fund_df["EPRatio"]= fund_df["EpsTTM"] / fund_df["Adjusted_close"]

#                 fund_df['Price_Ratio'] = fund_df['Adjusted_close'] / fund_df['Price_OED']
#                 fund_df['Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + fund_df['surprisePercent'])

#                 fund_df['Adj_YoY'] = fund_df['YoY'] * abs(fund_df['YoY'])
#                 fund_df['Adj_surprisePercent'] = (1/3000) * fund_df['surprisePercent'] * abs(fund_df['surprisePercent'])

#                 fund_df['alt_surprise'] = (fund_df['epsDifference'] / fund_df['Adjusted_close']) * 100

#                 fund_df['alt_Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + (fund_df['alt_surprise']/100))
#                 fund_df['SNP_Price_Ratio'] = fund_df['SNP_Close'] / fund_df['SNP_Price_OED']

#                 df_list.append(fund_df)
#             else:
#                 avoid.append(i)
#                 print(i)
#         except:
#             avoid.append(i)
#             print(i)

#     else:
#         print('No Fundamentals - {}'.format(i))
#         avoid.append(i)

# print('Completed>>>>>>1')    
        
# df_test = []

# for i in li:
#     aa = i.split('.')[0]
#     if (i != '.DS_Store') & (i not in avoid) & (i in tick) :
# #         if aa in al:


#         # df_test.append(temp)
#         try:
#             temp = pd.read_csv('/Volumes/AARTHI/sematic_data/testing_data/' + i)
#             temp['ticker'] = i

#             temp = helper.fundamental(aa, temp)
#             if (temp['YoYRev'].isnull().sum() == 0) & (temp['operatingMargin'].isnull().sum() == 0):
#                 temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#                 temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]

#                 fund_df = temp.merge(df_rate, on= ['Date']).drop(columns = ['Unnamed: 0'])
#                 fund_df = fund_df.merge(spx, on = 'Date')

#                 #Start with fundamental
#                 fund_df['Price_OED'] = None
#                 fund_df['SNP_Price_OED'] = None

#                 price_oed = None #Adjusted closing price on Earnings Day
#                 snp_price_oed = None #Adjusted closing price of S&P500 on Earnings Day
#                 prev_eps_seq = fund_df.iloc[0].eps_seq

#                 for i in range(fund_df.shape[0]):
#                     if (fund_df.iloc[i].event == 1.0):
#                         price_oed = fund_df.iloc[i].Adjusted_close
#                         snp_price_oed = fund_df.iloc[i].SNP_Close
#                         prev_eps_seq = fund_df.iloc[i].eps_seq
#                     elif (fund_df.iloc[i].eps_seq != prev_eps_seq):
#                         price_oed = fund_df.iloc[i].Adjusted_close
#                         snp_price_oed = fund_df.iloc[i].SNP_Close
#                         prev_eps_seq = fund_df.iloc[i].eps_seq
                        
#                     fund_df.iloc[i, fund_df.columns.get_loc('Price_OED')] = price_oed
#                     fund_df.iloc[i, fund_df.columns.get_loc('SNP_Price_OED')] = snp_price_oed

#                 fund_df["EpsTTM"] = fund_df["eps_actual"].apply(calcEpsTTM)
#                 fund_df["EPRatio"]= fund_df["EpsTTM"] / fund_df["Adjusted_close"]

#                 fund_df['Price_Ratio'] = fund_df['Adjusted_close'] / fund_df['Price_OED']
#                 fund_df['Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + fund_df['surprisePercent'])

#                 fund_df['Adj_YoY'] = fund_df['YoY'] * abs(fund_df['YoY'])
#                 fund_df['Adj_surprisePercent'] = (1/3000) * fund_df['surprisePercent'] * abs(fund_df['surprisePercent'])

#                 fund_df['alt_surprise'] = (fund_df['epsDifference'] / fund_df['Adjusted_close']) * 100

#                 fund_df['alt_Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + (fund_df['alt_surprise']/100))
#                 fund_df['SNP_Price_Ratio'] = fund_df['SNP_Close'] / fund_df['SNP_Price_OED']

#                 df_test.append(fund_df)
#             else:
#                 avoid.append(i)
#                 print(i)
#         except:
#             avoid.append(i)
#             print(i)

#         # temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#         # temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]
#         # df_test.append(temp)




# print('Completed>>>>>>2')    
        


# # df_val = []

# # for i in li:
# #     aa = i.split('.')[0]
# #     if (i != '.DS_Store') & (i not in avoid) & (i in tick) :
# # #         if aa in al:

# #         temp = pd.read_csv('validation_data/' + i)
# #         temp['ticker'] = i
# #         # df_test.append(temp)
# #         try:

# #             temp = helper.fundamental(aa, temp)
# #             if (temp['YoYRev'].isnull().sum() == 0) & (temp['operatingMargin'].isnull().sum() == 0):
# #                 temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
# #                 temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]

# #                 fund_df = temp.merge(df_rate, on= ['Date']).drop(columns = ['Unnamed: 0'])
# #                 fund_df = fund_df.merge(spx, on = 'Date')

# #                 #Start with fundamental
# #                 fund_df['Price_OED'] = None
# #                 fund_df['SNP_Price_OED'] = None

# #                 price_oed = None #Adjusted closing price on Earnings Day
# #                 snp_price_oed = None #Adjusted closing price of S&P500 on Earnings Day
# #                 prev_eps_seq = fund_df.iloc[0].eps_seq

# #                 for i in range(fund_df.shape[0]):
# #                     if (fund_df.iloc[i].event == 1.0):
# #                         price_oed = fund_df.iloc[i].Adjusted_close
# #                         snp_price_oed = fund_df.iloc[i].SNP_Close
# #                         prev_eps_seq = fund_df.iloc[i].eps_seq
# #                     elif (fund_df.iloc[i].eps_seq != prev_eps_seq):
# #                         price_oed = fund_df.iloc[i].Adjusted_close
# #                         snp_price_oed = fund_df.iloc[i].SNP_Close
# #                         prev_eps_seq = fund_df.iloc[i].eps_seq
                        
# #                     fund_df.iloc[i, fund_df.columns.get_loc('Price_OED')] = price_oed
# #                     fund_df.iloc[i, fund_df.columns.get_loc('SNP_Price_OED')] = snp_price_oed

# #                 fund_df["EpsTTM"] = fund_df["eps_actual"].apply(calcEpsTTM)
# #                 fund_df["EPRatio"]= fund_df["EpsTTM"] / fund_df["Adjusted_close"]

# #                 fund_df['Price_Ratio'] = fund_df['Adjusted_close'] / fund_df['Price_OED']
# #                 fund_df['Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + fund_df['surprisePercent'])

# #                 fund_df['Adj_YoY'] = fund_df['YoY'] * abs(fund_df['YoY'])
# #                 fund_df['Adj_surprisePercent'] = (1/3000) * fund_df['surprisePercent'] * abs(fund_df['surprisePercent'])

# #                 fund_df['alt_surprise'] = (fund_df['epsDifference'] / fund_df['Adjusted_close']) * 100

# #                 fund_df['alt_Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + (fund_df['alt_surprise']/100))
# #                 fund_df['SNP_Price_Ratio'] = fund_df['SNP_Close'] / fund_df['SNP_Price_OED']

# #                 df_val.append(fund_df)
# #             else:
# #                 avoid.append(i)
# #                 print(i)
# #         except:
# #             avoid.append(i)
# #             print(i)

# # #         # temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
# # #         # temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]
# # #         # df_test.append(temp)
        

# # rus = pd.read_csv('russel1000.csv')
# # rus['Tickers'] = rus['Tickers'] + '.csv'
# # ru = rus['Tickers'].unique().tolist()[:200]

# a = 'aapl_df, amzn_df, bb_df, clf_df, clne_df, fb_df, ibm_df, msft_df, nok_df, nvda_df,shop_df, sndl_df, tlry_df, tsla_df, wsbc_df, jpm_df, jnj_df, unh_df, v_df, pg_df, hd_df, pypl_df, dis_df, bac_df, ma_df,cmcsa_df, pfe_df, crm_df, nflx_df, csco_df, xom_df, vz_df, abt_df, tmo_df, ko_df,tmus_df, unp_df, hon_df, ups_df, ms_df, amgn_df, pm_df, c_df, bmy_df,lin_df, low_df, ba_df, sbux_df, chtr_df, intu_df, now_df, schw_df, blk_df, amd_df,t_df, mmm_df, bkng_df, gs_df, rtx_df, amat_df, amt_df, de_df, zts_df, isrg_df, adi_df, pld_df, chtr_df, tgt_df, ge_df, antm_df, axp_df, cvs_df, spgi_df, cat_df,syk_df, mo_df, gild_df, lrcx_df, cop_df, lmt_df, tjx_df, adp_df, mdlz_df, mu_df, pnc_df, mmc_df, usb_df, cci_df, cb_df, eqix_df, gm_df, duk_df, fis_df'


# # %%
# tickers = []
# for i in a.split(','):
#     tickers.append(i.split('_')[0].upper().replace(' ','') + '.csv')

tickers_ = tickers + ['AAPL.csv','FB.csv','AMZN.csv','MSFT.csv','TSLA.csv']
# tickers_ = tickers

# # print('Completed>>>>>>3')   
# new_df = pd.concat(df_list, ignore_index=True, sort=True)
# test_df = pd.concat(df_test, ignore_index=True, sort=True)
# new_df = new_df.reset_index(drop = True).reset_index()
# test_df = test_df.drop_duplicates().reset_index(drop = True)

# new_df_copy = pd.read_csv('/Volumes/AARTHI/sematic_data/train.csv')
# test_df_copy= pd.read_csv('/Volumes/AARTHI/sematic_data/test.csv')



# # # new_df = new_df[new_df['label'].isin([1,2])]
# new_df_copy = new_df_copy[new_df_copy['ticker'].isin(tickers_)]
# # val_df = val_df[val_df['ticker'].isin(tickers_)]



# # # new_df = new_df[new_df['label_3d'] != 0].reset_index(drop = True)
# # new_df = new_df.reset_index(drop = True).reset_index()
# # val_df = val_df.reset_index(drop = True).reset_index()

# # # new_df['label_3d'] = new_df['label_3d'].replace(2,0)



# # new_df = new_df.drop(columns = ['index'])
# # new_df = new_df.drop_duplicates().reset_index(drop = True)

# # val_df = val_df.drop(columns = ['index'])
# # val_df = val_df.drop_duplicates().reset_index(drop = True)

# # test_df = pd.concat(df_test, ignore_index=True, sort=True)
# # test_df = test_df.reset_index(drop = True).reset_index()
# test_df_copy = test_df_copy[test_df_copy['ticker'].isin(tickers_)]



# test_df = test_df.drop(columns = ['index'])
# test_df = test_df.drop_duplicates().reset_index(drop = True)
new_df = new_df_copy.copy()
test_df = test_df_copy.copy()

# new_df['direction_close'] = new_df['future_ch_p'].apply(direction)
# test_df['direction_close'] = test_df['future_ch_p'].apply(direction)



new_df['vol'] = new_df['future_ch_p'].apply(vola)
test_df['vol'] = test_df['future_ch_p'].apply(vola)

    

# new_df = new_df[new_df['direction_close'] != 2]
# test_df = test_df[test_df['direction_close'] != 2]

# new_df = new_df[new_df['label_4d'] != 2]
# val_df = val_df[val_df['label_4d'] != 2]
# test_df = test_df[test_df['label_4d'] != 2]

# new1 = new_df[new_df['direction_close'] == 1]
# new2 = new_df[new_df['direction_close'] == 0]
# new1 = new1.sample(n = len(new2))
# new_df = pd.concat([new1,new2]).reset_index(drop = True)
new_df = new_df.dropna().reset_index(drop = True)
test_df = test_df.dropna().reset_index(drop = True)



new_df = new_df.sample(frac=1).reset_index(drop=True)
# new_df = new_df.sample(frac=0.30).reset_index(drop = True)

# new_df = new_df[(new_df['Date'] <= '2020-02-01') | ((new_df['Date'] >= '2020-08-01') & (new_df['Date'] < '2021-07-01')) ].reset_index(drop = True)
new_df = new_df[(new_df['Date'] <= '2019-12-01') ].reset_index(drop = True)



# len(test_df)
# test_df = test_df[((test_df['Date'] >= '2021-07-01') &(test_df['Date'] <= '2021-12-01')) | ((test_df['Date'] >= '2019-09-01') &(test_df['Date'] <= '2019-12-31'))].reset_index(drop = True)
test_df = test_df[((test_df['Date'] >= '2021-07-01') & (test_df['Date'] < '2021-10-01')  )| ((test_df['Date'] < '2020-02-01') & (test_df['Date'] > '2019-12-01'))].reset_index(drop = True)

# test_df = test_df[(new_df['Date'] > '2021-01-01')].reset_index(drop = True).copy()
# new_df = new_df[new_df['Date'] <= '2020-01-01'].reset_index(drop = True)
# new_df = new_df[(new_df['Date'] <= '2021-07-01')].reset_index(drop = True)


# # len(test_df)
# # test_df = test_df[((test_df['Date'] >= '2021-07-01') &(test_df['Date'] <= '2021-12-01')) | ((test_df['Date'] >= '2019-01-01') &(test_df['Date'] <= '2020-01-31'))].reset_index(drop = True)
# test_df = test_df[(test_df['Date'] >= '2021-07-01') & (test_df['Date'] <= '2021-12-01')].reset_index(drop = True)
# len(test_df)
# new1 = new_df[new_df['vol'] == 1]
# new2 = new_df[new_df['vol'] == 0]
# new2 = new2.sample(frac = 0.8)
# new_df = pd.concat([new1,new2]).reset_index(drop = True)
new_df = new_df.dropna().reset_index(drop = True)

train_len = len(new_df)

new_df = new_df.append(test_df)



new_df = new_df.reset_index(drop = True)


price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []


    
price_lstm_np = np.array(price_lstm_).astype(np.float32)
# snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_).astype(np.float32)
pattern_lstm_np = np.array(pattern_lstm_).astype(np.float32)

# pattern_lstm_np = pattern_lstm_np - snp_price_lstm_np

test_price_lstm_np = price_lstm_np[train_len:]
price_lstm_np = price_lstm_np[:train_len]

test_volume_lstm_np = volume_lstm_np[train_len:]
volume_lstm_np = volume_lstm_np[:train_len]

test_pattern_lstm_np = pattern_lstm_np[train_len:]
pattern_lstm_np = pattern_lstm_np[:train_len]

# label_df = new_df['direction_close']
# new_df[:train_len].to_csv('train.csv',index = False)
# new_df[train_len:].to_csv('test.csv',index = False)

label_df = new_df['vol'][:train_len]
test_label_df = new_df['vol'][train_len:]

label_np = label_df.to_numpy().astype(np.float32)
test_label_np = test_label_df.to_numpy().astype(np.float32)
# val_label_np = val_label_df.to_numpy().astype(np.float32)

print('completed------2 \n')



# earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('weigts.h5', save_best_only=True, monitor='val_accuracy', mode='min')

# X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
# X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
# X_train3_df = new_df[[ '1d_DI+14_ushift',
#                       '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
#                       '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
#                       '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
#                       '3d_DI-20_ushift','7d_DI-20_ushift']] * 10


# X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10
# X_train4_df = (new_df[['ADX5', 'ADX9', 'ADX14', 'ADX20']] - 50)/50
# X_train5_df = new_df[[ 'DI+5','DI-5','DI+9','DI-9','DI+14','DI-14','DI+20','DI-20']]
# # X_train6_df = new_df[['YoY', 'surprisePercent', 'EPRatio']]
X_train6_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
                     '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
                      '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
                     '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
       'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift']]

# X_train6_df =  new_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p']]
# X_val_df =  val_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p']]



# X_train6_df = new_df[['macd259','signal259','hist259','YoY', 'surprisePercent', 'EPRatio', 'YoYRev', 'operatingMargin','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p','ADX14','DI+9','DI-9']]

# X_train3_df = new_df[['1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
#                       '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
#                      '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p']] * 10

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)

# X_train6_df = new_df[['7d_ch_p',  'vola9',  '2d_DI-20_ushift', 'VWMA2_ch_p',  '2d_DI+14_ushift', '2d_adx14_ch_p', '1d_RSI14_ch_p',  'surprisePercent', 'vola5',  'vola21', '7d_DI+14_ushift','2d_RSI14_ch_p', 'EPRatio']]
# X_train6_df = new_df[['7d_DI+14_ushift', 'surprisePercent', 'YoYRev', 'VWMA30_ch_p', '7d_DI-14_ushift', '2d_DI-20_ushift', 'YoY', '7d_adx9_ch_p', 'VWMA9_ch_p', '3d_adx9_ch_p', '21d_ch_v', '1d_DI+20_ushift', '12d_ch_v', '2d_DI+14_ushift', 'VWMA14_ch_p', '2d_ch_p']]
X_train_df = X_train6_df[:train_len]
X_test_df = X_train6_df[train_len:]
# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:]], axis=1)

# X_train_df = pd.concat([ X_train1_df,X_train5_df,X_train2_df,X_train4_df,X_train3_df,X_train6_df], axis=1)

# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train3_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:], X_train3_df[train_len:],X_train6_df[train_len:]], axis=1)


X_train_np = np.array(X_train_df).astype(np.float32)
X_test_np = np.array(X_test_df).astype(np.float32)
# X_val_np = np.array(X_val_df).astype(np.float32)


# define two sets of inputs

# lstm_seq_size = 40

# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(X_train_np, label_np, test_size=0.2)
# x_train, x_test, y_train, y_test = train_test_split(new_df[:train_len], label_np, test_size=0.2)

completed------2 



In [ ]:
X_train6_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
                     '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
                      '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
                     '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
       'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift']]

# X_train6_df =  new_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p']]
# X_val_df =  val_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p']]



# X_train6_df = new_df[['macd259','signal259','hist259','YoY', 'surprisePercent', 'EPRatio', 'YoYRev', 'operatingMargin','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p','ADX14','DI+9','DI-9']]

# X_train3_df = new_df[['1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
#                       '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
#                      '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p']] * 10

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)

# X_train6_df = new_df[['7d_ch_p',  'vola9',  '2d_DI-20_ushift', 'VWMA2_ch_p',  '2d_DI+14_ushift', '2d_adx14_ch_p', '1d_RSI14_ch_p',  'surprisePercent', 'vola5',  'vola21', '7d_DI+14_ushift','2d_RSI14_ch_p', 'EPRatio']]
# X_train6_df = new_df[['7d_DI+14_ushift', 'surprisePercent', 'YoYRev', 'VWMA30_ch_p', '7d_DI-14_ushift', '2d_DI-20_ushift', 'YoY', '7d_adx9_ch_p', 'VWMA9_ch_p', '3d_adx9_ch_p', '21d_ch_v', '1d_DI+20_ushift', '12d_ch_v', '2d_DI+14_ushift', 'VWMA14_ch_p', '2d_ch_p']]
X_train_df = X_train6_df[:train_len]
X_test_df = X_train6_df[train_len:]
# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:]], axis=1)

# X_train_df = pd.concat([ X_train1_df,X_train5_df,X_train2_df,X_train4_df,X_train3_df,X_train6_df], axis=1)

# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train3_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:], X_train3_df[train_len:],X_train6_df[train_len:]], axis=1)


X_train_np = np.array(X_train_df).astype(np.float32)
X_test_np = np.array(X_test_df).astype(np.float32)

In [17]:
from catboost import CatBoostClassifier, Pool

X_train_np_, X_val_np, label_np_, val_label_np = train_test_split(X_train_np, label_np, test_size=0.2)
test_data =  Pool(X_val_np, val_label_np)

model = CatBoostClassifier(
                            )

# model = CatBoostClassifier(
#                             )
# model = CatBoostClassifier(iterations=1000,
#                         depth=9,
#                         learning_rate=0.1,
#                         # loss_function='Logloss',
#                         eval_metric='AUC',
#                         )
# train the model
model.fit(X_train_np_, label_np_, eval_set = test_data, use_best_model=True, early_stopping_rounds=100)


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.141147
0:	learn: 0.6736102	test: 0.6738107	best: 0.6738107 (0)	total: 154ms	remaining: 2m 33s
1:	learn: 0.6590275	test: 0.6592661	best: 0.6592661 (1)	total: 337ms	remaining: 2m 47s
2:	learn: 0.6482324	test: 0.6485683	best: 0.6485683 (2)	total: 437ms	remaining: 2m 25s
3:	learn: 0.6402317	test: 0.6406275	best: 0.6406275 (3)	total: 535ms	remaining: 2m 13s
4:	learn: 0.6342532	test: 0.6347232	best: 0.6347232 (4)	total: 619ms	remaining: 2m 3s
5:	learn: 0.6297456	test: 0.6303249	best: 0.6303249 (5)	total: 691ms	remaining: 1m 54s
6:	learn: 0.6263753	test: 0.6269874	best: 0.6269874 (6)	total: 783ms	remaining: 1m 51s
7:	learn: 0.6237673	test: 0.6244390	best: 0.6244390 (7)	total: 854ms	remaining: 1m 45s
8:	learn: 0.6217159	test: 0.6224436	best: 0.6224436 (8)	total: 947ms	remaining: 1m 44s
9:	learn: 0.6201677	test: 0.6209203	best: 0.6209203 (9)	total: 1.03s	remaining: 1m 42s
10:	learn: 0.6189025	test: 0.6196829	best: 0.6196829 (10)	total: 1.11s	remaining: 1m 39s
11:	learn: 0

In [2226]:
# model.load_model('r1000_dir_selected_4p_5d_70_plus.model')

In [2303]:
# model.save_model('r1000_dir_selected_4p_5d_70_plus.model')

array([0.64547925, 0.71949004, 0.72141881, ..., 0.42343718, 0.58192811,
       0.51689271])

In [2382]:
# model.load_model('r1000_dir_selected_4p_5d_70_plus.model')

In [20]:

eval_data = X_test_np
preds_proba = model.predict_proba(eval_data)
# preds_class = model.predict(test_data)
pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.65 else 0)
pr_d['Date'] = test_df['Date']
o_d = []
for i in pr_d['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_d.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_d))

print(classification_report(test_label_np.tolist(), o_d))
# print(classification_report(test_label_np.tolist(), o_d))

[[18873   279]
 [ 6958   340]]
              precision    recall  f1-score   support

         0.0       0.73      0.99      0.84     19152
         1.0       0.55      0.05      0.09      7298

    accuracy                           0.73     26450
   macro avg       0.64      0.52      0.46     26450
weighted avg       0.68      0.73      0.63     26450



In [2355]:
pd.DataFrame({'columns': ['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
                     '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
                      '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
                     '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
       'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift','bop'],
'features': model.get_feature_importance().tolist()}).sort_values(by = 'features', ascending = False)

,columns,features
28,vola21,12.420661
27,vola9,6.072166
26,vola5,3.769851
23,7d_ch_v,2.803138
25,21d_ch_v,2.711591
21,VWMA30_ch_p,2.699248
51,bop,2.604392
24,12d_ch_v,2.518454
22,1d_ch_v,2.463212
16,2d_RSI14_ch_p,2.152331


In [2241]:
# model.save_model('r1000_dir_selected_4p_3d_70_plus.model')
# model_.save_model('r1000_vol_selected_4p_3d_70_plus.model')

In [2229]:
# model.load_model('r1000_dir_selected_4p_3d_70_plus.model')

In [1983]:
test_df['is_dir'] = o_d
temp_ = test_df[['ticker','is_dir','direction_close','label_4']]

In [1963]:
temp_['direction_down'] = temp_['direction_close'].replace(0,-1).replace(1,0).replace(-1,1)
temp_['is_down'] = temp_['is_dir'].replace(0,-1).replace(1,0).replace(-1,1)


In [1964]:
def check(x, y):
    if x == 1:
        if x == y:
            return 1
        else:
            return 0
    else:
        return 0

temp_['up_acc'] = temp_.apply(lambda row: check(row['is_dir'], row['direction_close']), axis = 1)
temp_['down_acc'] = temp_.apply(lambda row: check(row['is_down'], row['direction_down']), axis = 1)


In [530]:
eval_data = X_test_np_
preds_proba = model.predict_proba(eval_data)
pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.5 else 0)
o_d = []
for i in pr_d['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_d.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_d))

print(classification_report(test_label_np.tolist(), o_d))

[[ 111 1297]
 [  56  552]]
              precision    recall  f1-score   support

         0.0       0.66      0.08      0.14      1408
         1.0       0.30      0.91      0.45       608

    accuracy                           0.33      2016
   macro avg       0.48      0.49      0.30      2016
weighted avg       0.55      0.33      0.23      2016



In [2297]:
# new_df.to_csv('/Volumes/AARTHI/sematic_data/train.csv',index = False)
# test_df.to_csv('/Volumes/AARTHI/sematic_data/test.csv',index = False)

In [2291]:
pd.options.mode.chained_assignment = None

# tick = []
# stock_low = ['AMZN.csv','FB.csv','AAPL.csv','MSFT.csv','MMM.csv','AME.csv','ABBV.csv','ABT.csv','ADM.csv','AQN.csv','CAT.csv','CL.csv','CNP.csv','INTC.csv','JNJ.csv','GOOG.csv','JPM.csv','WMT.csv','PG.csv','V.csv','KO.csv', 'PEP.csv','UDP.csv','CL.csv','PG.csv','JPM.csv','CSCO.csv','MCD.csv','ADP.csv','SO.csv']

# stock_low = ['AMZN.csv','FB.csv','AAPL.csv','MSFT.csv','GOOG.csv']



    

# av = ['OGN.csv', 'BF.csv', 'BRK.csv', 'CARR.csv', 'OTIS.csv','BRK-A.csv','MRNA.csv','TSLA.csv','NVDA.csv']
# av = ['OGN.csv', '.DS_Store', 'BF.csv', 'BRK.csv']
# # # # av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
# for i in os.listdir('training_data'):
# #     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
#     if (i != '.ipynb_checkpoints') and (i not in av):
#         tick.append(i)


# df_list = []

# # for i in tick:
# for i in stock_low:
#     try: 
#         aa = i.split('.')[0]
#     #     if aa in al:

#         temp = pd.read_csv('training_data/' + i)
#         temp['ticker'] = i
#     #     temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#     #     temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]
#         df_list.append(temp)
#     except:
#         print(i)
    
# # stock_low = ['AMZN.csv','FB.csv','AAPL.csv','MSFT.csv','MMM.csv','AME.csv','ABBV.csv','ABT.csv','ADM.csv','AQN.csv','CAT.csv','CL.csv','CNP.csv','INTC.csv','JNJ.csv','GOOG.csv','JPM.csv','WMT.csv','PG.csv','V.csv','KO.csv', 'PEP.csv','UDP.csv','CL.csv','PG.csv','JPM.csv','CSCO.csv','MCD.csv','ADP.csv','SO.csv']
      
# df_test = []

# # for i in tick:
# for i in stock_low:
#     try:
#         aa = i.split('.')[0]
#         if i != '.DS_Store':
#     #         if aa in al:

#             temp = pd.read_csv('testing_data/' + i)
#             temp['ticker'] = i
#     #         temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#     #         temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]
#             df_test.append(temp)
#     except:
#         print(i)



        
# new_df = pd.concat(df_list, ignore_index=True, sort=True)
# # new_df = new_df[new_df['ticker'].isin(stock_low)]
# # new_df = new_df[new_df['label'].isin([1,2])]


# # new_df = new_df[new_df['label_3d'] != 0].reset_index(drop = True)
# new_df = new_df.reset_index(drop = True).reset_index()

# new_df['label_3d'] = new_df['label_3d'].replace(2,0)
tickers_ = tickers + ['AAPL.csv','FB.csv','AMZN.csv','MSFT.csv','TSLA.csv']
# tickers_ = tickers

# print('Completed>>>>>>3')   
new_df = pd.concat(df_list, ignore_index=True, sort=True)
# val_df = pd.concat(df_val, ignore_index=True, sort=True)
# new_df = new_df[new_df['label'].isin([1,2])]
new_df = new_df[new_df['ticker'].isin(tickers_)]
# val_df = val_df[val_df['ticker'].isin(tickers_)]


# new_df = new_df.drop(columns = ['index'])
new_df = new_df.drop_duplicates().reset_index(drop = True)

test_df = pd.concat(df_test, ignore_index=True, sort=True)
test_df = test_df[test_df['ticker'].isin(tickers_)]
# test_df = test_df[test_df['ticker'].isin(stock_low)]
test_df = test_df.reset_index(drop = True).reset_index()



# test_df = test_df.drop(columns = ['index'])
test_df = test_df.drop_duplicates().reset_index(drop = True)




# new_df['direction_close'] = new_df['future_ch_p'].apply(direction)
# test_df['direction_close'] = test_df['future_ch_p'].apply(direction)

new_df['vol'] = new_df['future_ch_p3'].apply(vola)
test_df['vol'] = test_df['future_ch_p3'].apply(vola)

new_df['direction_close'] = new_df['future_ch_p3'].apply(direction)
# val_df['direction_close'] = val_df['future_ch_p'].apply(direction)
test_df['direction_close'] = test_df['future_ch_p3'].apply(direction)

new_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
new_df = new_df.dropna().reset_index(drop = True)
test_df = test_df.dropna().reset_index(drop = True)

# new_df = new_df.reset_index(drop = True).reset_index()



# new_df = new_df.merge(snp_train, on = 'Date')
# test_df = test_df.merge(snp_test, on = 'Date')

# new_df["latest_change"] = new_df["future_ch_p"] - new_df["snp_future_ch_p"]
# test_df["latest_change"] = test_df["future_ch_p"] - test_df["snp_future_ch_p"]

# new_df["3l"]=new_df["latest_change"].apply(add3LIndicator_2D)
# new_df["label"] = new_df["3l"].apply(addLabel)

# test_df["3l"]=test_df["latest_change"].apply(add3LIndicator_2D)
# test_df["label"] = test_df["3l"].apply(addLabel)

# new_df = new_df.sample(frac=1).reset_index(drop=True)
# new1 = new_df[new_df['vol'] == 1]
# new2 = new_df[new_df['vol'] == 0]
# new2 = new2.sample(frac = 0.4)
# new_df = pd.concat([new1,new2]).reset_index(drop = True)
new_df = new_df.dropna().reset_index(drop = True)
# new_df = new_df.sample(frac=0.30).reset_index(drop = True)

# test_df = test_df[(new_df['Date'] > '2021-01-01')].reset_index(drop = True).copy()
# new_df = new_df[new_df['Date'] <= '2020-01-01'].reset_index(drop = True)
new_df = new_df[(new_df['Date'] <= '2021-07-01') ].reset_index(drop = True)

# len(test_df)
test_df = test_df[(test_df['Date'] >= '2021-07-01') &(test_df['Date'] <= '2021-12-07')].reset_index(drop = True)



train_len = len(new_df)

new_df = new_df.append(test_df)



new_df = new_df.reset_index(drop = True)

# price_df = new_df["Price_Seq"]
# # snp_price_df = new_df["snp_Change_Seq"]
# volume_df = new_df["Volume_Seq"]
# change_df = new_df['Change_Seq']

price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []


    
price_lstm_np = np.array(price_lstm_).astype(np.float32)
# snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_).astype(np.float32)
pattern_lstm_np = np.array(pattern_lstm_).astype(np.float32)

# pattern_lstm_np = pattern_lstm_np - snp_price_lstm_np

test_price_lstm_np = price_lstm_np[train_len:]
price_lstm_np = price_lstm_np[:train_len]

test_volume_lstm_np = volume_lstm_np[train_len:]
volume_lstm_np = volume_lstm_np[:train_len]

test_pattern_lstm_np = pattern_lstm_np[train_len:]
pattern_lstm_np = pattern_lstm_np[:train_len]

# label_df = new_df['direction_close']

# new_df[:train_len].to_csv('train.csv',index = False)
# new_df[train_len:].to_csv('test.csv',index = False)

label_df = new_df['vol'][:train_len]
test_label_df = new_df['vol'][train_len:]
test_label_df_ = new_df['label_4'][train_len:]
label_np = label_df.to_numpy().astype(np.float32)
test_label_np = test_label_df.to_numpy().astype(np.float32)
test_label_np_ = test_label_df_.to_numpy().astype(np.float32)

print('completed------2 \n')



# earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('weigts.h5', save_best_only=True, monitor='val_accuracy', mode='min')

X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[[ '1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift']] * 10


X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10
X_train4_df = (new_df[['ADX5', 'ADX9', 'ADX14', 'ADX20']] - 50)/50
X_train5_df = new_df[[ 'DI+5','DI-5','DI+9','DI-9','DI+14','DI-14','DI+20','DI-20']]
# X_train6_df = new_df[['YoY', 'surprisePercent', 'EPRatio']]
X_train6_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
                     '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
                      '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
                     '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
       'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift','bop']]
# X_train6_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
#                      '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
#                       '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
#                      '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
#        'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p','bop']]

X_train6_df_ =  new_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p']]
# X_train6_df = new_df[['alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', '1d_ch_p',  '7d_ch_p','operatingMargin','stoch_d','FR_3M_ch_p', '1d_vol',
#  '12d_vol',
#  '21d_vol',
#  '7d_vol','1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v']]

# X_train6_df = new_df[['12d_ch_v','1d_ch_v', 'VWMA20_ch_p']]
# X_train6_df = new_df[['7d_DI+14_ushift', 'surprisePercent', 'YoYRev', 'VWMA30_ch_p', '7d_DI-14_ushift', '2d_DI-20_ushift', 'YoY', '7d_adx9_ch_p', 'VWMA9_ch_p', '3d_adx9_ch_p', '21d_ch_v', '1d_DI+20_ushift', '12d_ch_v', '2d_DI+14_ushift', 'VWMA14_ch_p', '2d_ch_p']]
X_train_df_ = X_train6_df_[:train_len]
X_test_df_ = X_train6_df_[train_len:]


# X_train6_df = new_df[['12d_ch_v','1d_ch_v', 'VWMA20_ch_p']]
# X_train6_df = new_df[['7d_DI+14_ushift', 'surprisePercent', 'YoYRev', 'VWMA30_ch_p', '7d_DI-14_ushift', '2d_DI-20_ushift', 'YoY', '7d_adx9_ch_p', 'VWMA9_ch_p', '3d_adx9_ch_p', '21d_ch_v', '1d_DI+20_ushift', '12d_ch_v', '2d_DI+14_ushift', 'VWMA14_ch_p', '2d_ch_p']]
X_train_df = X_train6_df[:train_len]
X_test_df = X_train6_df[train_len:]

# X_train3_df = new_df[['1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
#                       '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
#                      '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p']] * 10

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)


# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:],X_train6_df[train_len:]], axis=1)

# X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len],X_train4_df[:train_len],X_train5_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:],X_train4_df[train_len:],X_train5_df[train_len:]], axis=1)

# X_train_df = pd.concat([ X_train1_df,X_train5_df,X_train2_df,X_train4_df,X_train3_df,X_train6_df], axis=1)

# X_train_df = pd.concat([ X_train3_df[:train_len],X_train5_df[:train_len],X_train3_df[:train_len],X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([ X_train3_df[train_len:],X_train5_df[train_len:], X_train3_df[train_len:],X_train6_df[train_len:]], axis=1)
# X_train_df.to_csv('train.csv',index = False)
# X_test_df.to_csv('test.csv',index = False)


# X_train_np = np.array(X_train_df).astype(np.float32)
# X_test_np = np.array(X_test_df).astype(np.float32)

X_train_np = np.array(X_train_df)
X_test_np = np.array(X_test_df)

# X_train_np_ = np.array(X_train_df_).astype(np.float32)
# X_test_np_ = np.array(X_test_df_).astype(np.float32)



completed------2 



In [2292]:

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
# import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
# import statsmodels.api as sm
from sklearn import linear_model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from vecstack import stacking
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import PowerTransformer
from sklearn.tree import DecisionTreeRegressor  
from sklearn import svm
from scipy.stats import norm, skew #for some statistics
from sklearn.linear_model import ElasticNet
# from lightgbm import *
from sklearn.metrics import mean_squared_error
from vecstack import StackingTransformer
# from xgboost import *
from sklearn.metrics import *
import plotly.express as px
import catboost
import time
from datetime import datetime, timedelta
from sklearn.metrics import *
import optuna
import random
from collections import OrderedDict

rf = ('rf',RandomForestClassifier(random_state=0
                                                , n_jobs=-1
                                                , n_estimators=100
                                                , max_depth=6))
cat = ('cat',catboost.CatBoostClassifier(#booster = 'gbtree'
                                        #, colsample_bytree = 0.7000000000000001
                                        #, eval_metric = 'rmse'
                                        #, gamma = 3.6026764795843857
                                        #, learning_rate = 0.21782414954966559 
                                        #, max_depth = 5
                                        #, min_child_weight = 5.510736763551528
                                        #, n_estimators = 225.7931274292029
                                            verbose = False, depth = 8,
                                        ))

ridge = ('ridge', Ridge(alpha=1.0
                                    , copy_X=True
                                    , fit_intercept=True
                                    , max_iter=None
                                    , normalize=False
                                    , random_state=None
                                    , solver='auto'
                                    , tol=0.001))


en = ('en', ElasticNet(alpha=1.0
                                        , copy_X=True
                                        , fit_intercept=True
                                        , l1_ratio=0.5
                                        , max_iter=1000
                                        , normalize=False
                                        , positive=False
                                        , precompute=False
                                        , random_state=0
                                        , selection='cyclic'
                                        , tol=0.0001
                                        , warm_start=False))



    
    
estimators_l1= [cat, rf]

stack = StackingTransformer(estimators=estimators_l1,   # base estimators
                        regression=True,            # regression task (set to False for classification)
                        variant='A',                # oof for train set, predict test 
                                                    #     set in each fold and find mean
                        metric=mean_squared_error, # metric: callable
                        n_folds=4,                  # number of folds
                        shuffle=True,               # shuffle the data
                        random_state=1,             # ensure reproducibility
                        verbose=2) 

l2_model = catboost.CatBoostClassifier(#booster = 'gbtree'
                                        #, colsample_bytree = 0.7000000000000001
                                        #, eval_metric = 'rmse'
                                        #, gamma = 3.6026764795843857
                                        #, learning_rate = 0.21782414954966559 
                                        #, max_depth = 5
                                        #, min_child_weight = 5.510736763551528
                                        #, n_estimators = 225.7931274292029
                                            verbose = False, depth = 8,
                                        )



stack = stack.fit(X_train_np, label_np)


S_train = stack.transform(X_train_np)
l2_model = l2_model.fit(S_train, label_np)

S_test = stack.transform(X_test_np)
y_pred = l2_model.predict_proba(S_test)




    



task:         [regression]
metric:       [mean_squared_error]
variant:      [A]
n_estimators: [2]

estimator  0: [cat: CatBoostClassifier]
    fold  0:  [0.23603764]
    fold  1:  [0.23567233]
    fold  2:  [0.23387839]
    fold  3:  [0.23408297]
    ----
    MEAN:     [0.23491783] + [0.00094877]

estimator  1: [rf: RandomForestClassifier]
    fold  0:  [0.24454219]
    fold  1:  [0.24588655]
    fold  2:  [0.24624085]
    fold  3:  [0.24473573]
    ----
    MEAN:     [0.24535133] + [0.00072653]

Train set was detected.
Transforming...

estimator  0: [cat: CatBoostClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [rf: RandomForestClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [cat: CatBoostClassifier]
    model from fold  0: done
    model from fol

In [2271]:
y_pred

array([0.50002877, 0.50002877, 0.50002877, ..., 0.50002877, 0.50002877,
       0.50002877])

In [2272]:
# import torch 

# x_train, x_test, y_train, y_test = train_test_split(X_train_np, label_np, test_size=0.1)

# TabNetPretrainer
# unsupervised_model = TabNetPretrainer(
#     optimizer_fn=torch.optim.Adam,
#     optimizer_params=dict(lr=1e-2),
#     mask_type='entmax' # "sparsemax"
# )

# unsupervised_model.fit(
#     X_train=x_train,
#     eval_set=[x_test],
#     pretraining_ratio=0.8,
# )

# clf = TabNetClassifier(
#     optimizer_fn=torch.optim.Adam,
#     optimizer_params=dict(lr=1e-2),
#     scheduler_params={"step_size":10, # how to use learning rate scheduler
#                       "gamma":0.9},
#     scheduler_fn=torch.optim.lr_scheduler.StepLR,
#     mask_type='sparsemax' # This will be overwritten if using pretrain model
# )

# clf.fit(
#     X_train=x_train, y_train=y_train,
#     eval_set=[(x_train, y_train), (x_test, y_test)],
#     eval_name=['train', 'valid'],
#     eval_metric=['auc'],
#     from_unsupervised=unsupervised_model
# )

In [2236]:
from catboost import CatBoostClassifier, Pool
x_train, x_test, y_train, y_test = train_test_split(X_train_np, label_np, test_size=0.3)

cat_features = [0]

test_data =  Pool(x_test, y_test)

model_ = CatBoostClassifier(iterations=1000,
                            depth=7,
                            learning_rate=0.126,
                            random_strength = 5,
                            bagging_temperature = 0.6682247139682966,
                            l2_leaf_reg = 28,
                            border_count = 197, 
                            loss_function='Logloss',
                            eval_metric='AUC'
                            )



# model_ = CatBoostClassifier(iterations=1000,
#                         depth=8,
#                         learning_rate=0.003,
#                         verbose = True)


# train the model
model_.fit(x_train, y_train,eval_set = test_data )




0:	test: 0.7089370	best: 0.7089370 (0)	total: 41.7ms	remaining: 41.7s
1:	test: 0.7244372	best: 0.7244372 (1)	total: 71.9ms	remaining: 35.9s
2:	test: 0.7316743	best: 0.7316743 (2)	total: 99.3ms	remaining: 33s
3:	test: 0.7317615	best: 0.7317615 (3)	total: 127ms	remaining: 31.5s
4:	test: 0.7335348	best: 0.7335348 (4)	total: 156ms	remaining: 30.9s
5:	test: 0.7343630	best: 0.7343630 (5)	total: 186ms	remaining: 30.8s
6:	test: 0.7365471	best: 0.7365471 (6)	total: 213ms	remaining: 30.3s
7:	test: 0.7367019	best: 0.7367019 (7)	total: 244ms	remaining: 30.2s
8:	test: 0.7369260	best: 0.7369260 (8)	total: 273ms	remaining: 30.1s
9:	test: 0.7374915	best: 0.7374915 (9)	total: 303ms	remaining: 30s
10:	test: 0.7372922	best: 0.7374915 (9)	total: 333ms	remaining: 29.9s
11:	test: 0.7375461	best: 0.7375461 (11)	total: 365ms	remaining: 30s
12:	test: 0.7384025	best: 0.7384025 (12)	total: 399ms	remaining: 30.3s
13:	test: 0.7386411	best: 0.7386411 (13)	total: 433ms	remaining: 30.5s
14:	test: 0.7389162	best: 0.73

In [2238]:
eval_data = X_test_np
preds_proba = model_.predict_proba(eval_data)
pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.65 else 0)
o = []
for i in pr_d['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[29610   199]
 [ 9762   280]]
              precision    recall  f1-score   support

         0.0       0.75      0.99      0.86     29809
         1.0       0.58      0.03      0.05     10042

    accuracy                           0.75     39851
   macro avg       0.67      0.51      0.45     39851
weighted avg       0.71      0.75      0.65     39851



In [2280]:
y_pred = l2_model.predict_proba(S_test)

In [2294]:
eval_data = X_test_np
preds_proba = l2_model.predict_proba(S_test)
pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.6 else 0)
o = []
for i in pr_d['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[29526   283]
 [ 9663   379]]
              precision    recall  f1-score   support

         0.0       0.75      0.99      0.86     29809
         1.0       0.57      0.04      0.07     10042

    accuracy                           0.75     39851
   macro avg       0.66      0.51      0.46     39851
weighted avg       0.71      0.75      0.66     39851



In [2279]:
y_pred

array([1., 1., 0., ..., 1., 1., 1.])

In [2239]:
model_4v = CatBoostClassifier(iterations=500,
                            depth=7,
                            learning_rate=0.126,
                            random_strength = 5,
                            bagging_temperature = 0.6682247139682966,
                            l2_leaf_reg = 28,
                            border_count = 197, 
                            loss_function='Logloss',
                            eval_metric='AUC'
                            )

model_4v.load_model('r1000_vol_selected_4p_5d_70_plus.model')

In [2240]:
model_4d = CatBoostClassifier(iterations=500,
                            depth=7,
                            learning_rate=0.126,
                            random_strength = 5,
                            bagging_temperature = 0.6682247139682966,
                            l2_leaf_reg = 28,
                            border_count = 197, 
                            loss_function='Logloss',
                            eval_metric='AUC'
                            )

model_4d.load_model('r1000_dir_selected_4p_5d_70_plus.model')

In [2243]:
X_test_np_ = np.array(X_test_df_)
eval_data = X_test_np_
preds_proba = model_4d.predict_proba(eval_data)
pr_d4 = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d4['pred'] = pr_d4['val'].apply(lambda x: 1 if x >= 0.5 else 0)
o_4d = []
for i in pr_d4['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_4d.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_4d))

print(classification_report(test_label_np.tolist(), o_4d))

[[10985 18824]
 [ 3807  6235]]
              precision    recall  f1-score   support

         0.0       0.74      0.37      0.49     29809
         1.0       0.25      0.62      0.36     10042

    accuracy                           0.43     39851
   macro avg       0.50      0.49      0.42     39851
weighted avg       0.62      0.43      0.46     39851



In [2244]:
eval_data = X_test_np
preds_proba = model_4v.predict_proba(eval_data)
pr_v4 = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_v4['pred'] = pr_d4['val'].apply(lambda x: 1 if x >= 0.5 else 0)
o_4v = []
for i in pr_v4['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_4v.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_4v))

print(classification_report(test_label_np.tolist(), o_4v))

[[10985 18824]
 [ 3807  6235]]
              precision    recall  f1-score   support

         0.0       0.74      0.37      0.49     29809
         1.0       0.25      0.62      0.36     10042

    accuracy                           0.43     39851
   macro avg       0.50      0.49      0.42     39851
weighted avg       0.62      0.43      0.46     39851



In [2245]:
X_test_np_ = np.array(X_test_df_)
eval_data = X_test_np_
preds_proba = model.predict_proba(eval_data)
pr_= pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_['pred'] = pr_['val'].apply(lambda x: 1 if x >= 0.5 else 0)
o_d = []
for i in pr_['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_d.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_d))

print(classification_report(test_label_np.tolist(), o_d))

[[13031 16778]
 [ 4219  5823]]
              precision    recall  f1-score   support

         0.0       0.76      0.44      0.55     29809
         1.0       0.26      0.58      0.36     10042

    accuracy                           0.47     39851
   macro avg       0.51      0.51      0.46     39851
weighted avg       0.63      0.47      0.50     39851



In [2006]:
# eval_data = X_test_np
# preds_proba = clf.predict_proba(eval_data)
# pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
# pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.35 else 0)
# o = []
# for i in pr_d['pred'].tolist():
# #     if i[1] > 0.7:''
# #         o.append(1)z
# #     else:
# #         o.append(o)
    
#     o.append(i)
    
# print(confusion_matrix(test_label_np.tolist(), o))

# print(classification_report(test_label_np.tolist(), o))

In [2007]:
# model_.save_model('r1000_selected_2p_3d.model')

In [2008]:
print(confusion_matrix(test_label_np_.tolist(), o))
print(classification_report(test_label_np_.tolist(), o))

[[19493   319]
 [16480  1563]]
              precision    recall  f1-score   support

         0.0       0.54      0.98      0.70     19812
         1.0       0.83      0.09      0.16     18043

    accuracy                           0.56     37855
   macro avg       0.69      0.54      0.43     37855
weighted avg       0.68      0.56      0.44     37855



In [2246]:
test_df.columns.tolist()

['index',
 '12d_ch_v',
 '12d_vol',
 '1d_DI+14_ch_p',
 '1d_DI+14_ushift',
 '1d_DI+20_ch_p',
 '1d_DI+20_ushift',
 '1d_DI-14_ch_p',
 '1d_DI-14_ushift',
 '1d_DI-20_ch_p',
 '1d_DI-20_ushift',
 '1d_RSI14_ch_p',
 '1d_RSI14_ushift',
 '1d_adx14_ch_p',
 '1d_adx14_ushift',
 '1d_adx9_ch_p',
 '1d_adx9_ushift',
 '1d_ch_p',
 '1d_ch_v',
 '1d_ushift',
 '1d_vol',
 '1ushift',
 '1ushift_h',
 '1ushift_l',
 '1ushift_o',
 '1ushift_v',
 '21d_ch_v',
 '21d_vol',
 '2d_DI+14_ch_p',
 '2d_DI+14_ushift',
 '2d_DI+20_ch_p',
 '2d_DI+20_ushift',
 '2d_DI-14_ch_p',
 '2d_DI-14_ushift',
 '2d_DI-20_ch_p',
 '2d_DI-20_ushift',
 '2d_RSI14_ch_p',
 '2d_RSI14_ushift',
 '2d_adx14_ch_p',
 '2d_adx14_ushift',
 '2d_adx9_ch_p',
 '2d_adx9_ushift',
 '2d_ch_p',
 '2d_ushift',
 '3_day_fed_fund_rolling',
 '3d_DI+14_ch_p',
 '3d_DI+14_ushift',
 '3d_DI+20_ch_p',
 '3d_DI+20_ushift',
 '3d_DI-14_ch_p',
 '3d_DI-14_ushift',
 '3d_DI-20_ch_p',
 '3d_DI-20_ushift',
 '3d_RSI14_ch_p',
 '3d_RSI14_ushift',
 '3d_adx14_ch_p',
 '3d_adx14_ushift',
 '3d_adx9_ch_p

In [2251]:
test_df['is_vol'] = o
test_df['is_dir'] = o_d
test_df['is_vol4'] = o_4v
test_df['is_dir4'] = o_4d
test_df['volatality_prob'] = pr_d['val'].tolist()
test_df['Direction_prob'] = pr_['val'].tolist()
test_df['volatality_prob_4'] = pr_v4['val'].tolist()
test_df['Direction_prob_4'] = pr_d4['val'].tolist()
temp_ = test_df[['ticker','is_vol','label_2','vol', 'direction_close','is_dir', 'max_ch','min_ch', 'future_ch_p']]
temp_plot = test_df[['ticker','is_vol','is_dir','volatality_prob', 'Direction_prob', 'max_ch','min_ch', 'future_ch_p','Date', 'volatality_prob_4','Direction_prob_4','is_vol4','is_dir4',
 'future_ch_p1',
 'future_ch_p2',
 'future_ch_p3',
 'future_ch_p4', 'min_ch_1',
 'min_ch_1.1',
 'min_ch_2',
 'min_ch_2.1',
 'min_ch_3',
 'min_ch_3.1',
 'min_ch_4',
 'min_ch_4.1',
 'min_ch_5',
 'min_ch_5.1']]
temp_ =temp_[temp_['is_vol'] == 1]


In [2253]:
temp_plot.columns = ['ticker', 'is_vol', 'is_dir', 'volatality_prob', 'Direction_prob',
       'max_ch', 'min_ch', 'future_ch_p', 'Date', 'volatality_prob_4',
       'Direction_prob_4', 'is_vol4', 'is_dir4', 'future_ch_p1',
       'future_ch_p2', 'future_ch_p3', 'future_ch_p4', 'max_ch_1',
       'min_ch_1', 'max_ch_2', 'min_ch_2', 'max_ch_3', 'min_ch_3',
       'max_ch_4', 'min_ch_4', 'max_ch_5', 'min_ch_5']

In [2254]:
temp_plot_up = temp_plot[temp_plot['is_dir'] == 1]
temp_plot_down = temp_plot[temp_plot['is_dir'] == 0]

In [2255]:
for i in ['max_ch','min_ch', 'future_ch_p','future_ch_p1','future_ch_p2','future_ch_p3','future_ch_p4']:
    temp_plot_up[i] = temp_plot_up[i] * 100
    temp_plot_down[i] = temp_plot_down[i] * 100

In [2256]:
temp_plot_up['Success'] = temp_plot_up['future_ch_p'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
temp_plot_down['Success'] = temp_plot_down['future_ch_p'].apply(lambda x: 'Positive' if x < 0 else 'Negative')
temp_plot_up['Success1'] = temp_plot_up['future_ch_p1'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
temp_plot_down['Success1'] = temp_plot_down['future_ch_p1'].apply(lambda x: 'Positive' if x < 0 else 'Negative')
temp_plot_up['Success2'] = temp_plot_up['future_ch_p2'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
temp_plot_down['Success2'] = temp_plot_down['future_ch_p2'].apply(lambda x: 'Positive' if x < 0 else 'Negative')
temp_plot_up['Success3'] = temp_plot_up['future_ch_p3'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
temp_plot_down['Success3'] = temp_plot_down['future_ch_p3'].apply(lambda x: 'Positive' if x < 0 else 'Negative')
temp_plot_up['Success4'] = temp_plot_up['future_ch_p4'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
temp_plot_down['Success4'] = temp_plot_down['future_ch_p4'].apply(lambda x: 'Positive' if x < 0 else 'Negative')

In [2257]:
temp_plot_up.to_csv('5d_scatter_up.csv',index = False)
temp_plot_down.to_csv('5d_scatter_down.csv',index = False)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Direction_prob=%{x}<br>future_ch_p=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([0.6799317 , 0.65609073, 0.68723537, ..., 0.52825702, 0.58832894,
                          0.69202629]),
              'xaxis': 'x',
              'y': array([-2.87322086, -1.26719523,  0.92731824, ..., -3.648473  , -1.67412101,
                          -6.19616047]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Direction_prob'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'future_ch_p'}}}
})

In [1859]:
def check_up(x,y):
    if x == 1:
        if y >= 0.02:
            return 1
        else:
            return 0
    else:
        return 0
def check_down(x,y):
    if x == 0:
        if y <= -0.02:
            return 1
        else:
            return 0
    else:
        return 0

temp_['up'] = temp_.apply(lambda row: check_up(row['is_dir'], row['max_ch']),axis = 1)
temp_['down'] = temp_.apply(lambda row: check_down(row['is_dir'], row['min_ch']), axis = 1)

In [1860]:
118/140

0.8428571428571429

In [1861]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.02)].shape

(4982, 11)

In [1862]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.04)].shape

(2492, 11)

In [1863]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.04)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.2895317764610201

In [1864]:
temp_[(temp_['is_dir'] == 1) & (temp_['future_ch_p'] >= 0.02)].shape[0]

2771

In [1865]:

temp_[(temp_['is_dir'] == 1)].shape[0]

8607

In [1866]:
temp_[(temp_['is_dir'] == 1) & (temp_['future_ch_p'] >= 0.02)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.32194725223655163

In [1868]:
temp_[(temp_['is_dir'] == 1) & (temp_['future_ch_p'] >= 0.01)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.4227953990937609

In [1869]:
temp_[(temp_['is_dir'] == 1)].shape[0]

8607

In [1870]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.03)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.41466248402463113

In [1871]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.03) & (temp_['min_ch'] >= -0.03)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.37353317067503194

In [1872]:
temp_[(temp_['is_dir'] == 1) & (temp_['max_ch'] >= 0.02)].shape[0]/temp_[(temp_['is_dir'] == 1)].shape[0]

0.578831183920065

In [1873]:
temp_[(temp_['is_dir'] == 0) ].shape

(4104, 11)

In [1874]:
temp_[(temp_['is_dir'] == 0) & (temp_['min_ch'] <= -0.02)].shape

(2508, 11)

In [1875]:
temp_[(temp_['is_dir'] == 0) & (temp_['future_ch_p'] <= -0.02)].shape

(1314, 11)

In [1878]:
1314/4104

0.3201754385964912

In [1848]:
temp_[(temp_['is_dir'] == 0) & (temp_['min_ch'] <= -0.04)].shape

(26, 11)

In [1849]:
temp_[(temp_['is_dir'] == 0) & (temp_['min_ch'] <= -0.02)].shape[0]/temp_[(temp_['is_dir'] == 0) ].shape[0]

0.6933333333333334

In [1850]:
temp_[(temp_['is_dir'] == 0) & (temp_['min_ch'] <= -0.03)].shape[0]/temp_[(temp_['is_dir'] == 0) ].shape[0]

0.56

In [1851]:
temp_[(temp_['is_dir'] == 0) & (temp_['min_ch'] <= -0.04)].shape[0]/temp_[(temp_['is_dir'] == 0) ].shape[0]

0.3466666666666667

In [1852]:
temp_['ticker'].nunique()

63

In [1625]:
temp_['ticker'].value_counts().reset_index()

,index,ticker
0,APA.csv,80
1,EQT.csv,80
2,AAL.csv,79
3,PLUG.csv,79
4,CLF.csv,79
...,...,...
645,VVV.csv,1
646,DVA.csv,1
647,BXP.csv,1
648,UHAL.csv,1


In [1626]:
up_ = temp_.groupby('ticker')[['is_dir','up']].sum().reset_index()
up_['acc'] = up_['up']/up_['is_dir']


In [1627]:
t = temp_.copy()
t['is_dir'] = t['is_dir'].replace(1,2).replace(0,1).replace(2,0)
down_ = t.groupby('ticker')[['is_dir','down']].sum().reset_index()
down_['acc'] = down_['down']/down_['is_dir']


In [1628]:
up_[up_['acc'] > 0.4].sort_values('acc', ascending = False)['acc'].mean()

0.619635259910989

In [1629]:
up_[up_['acc'] > 0.5].sort_values('acc', ascending = False)[['is_dir','up']].sum()

is_dir    3169
up        1967
dtype: int64

In [1630]:
up_[up_['acc'] > 0.5].sort_values('acc', ascending = False)

,ticker,is_dir,up,acc
353,MASI.csv,1,1,1.000000
87,BXP.csv,1,1,1.000000
110,CFX.csv,2,2,1.000000
114,CHE.csv,3,3,1.000000
458,PNR.csv,1,1,1.000000
...,...,...,...,...
317,KSS.csv,54,28,0.518519
151,CZR.csv,27,14,0.518519
9,ADPT.csv,62,32,0.516129
4,ACI.csv,33,17,0.515152


In [1643]:
up_[up_['acc'] > 0.7].sort_values('acc', ascending = False)['ticker'].nunique()

67

In [1644]:
down_[down_['acc'] > 0.7].sort_values('acc', ascending = False)['ticker'].nunique()

52

In [1645]:
all = up_[up_['acc'] > 0.7].sort_values('acc', ascending = False)[['ticker','acc']].append(down_[down_['acc'] > 0.7].sort_values('acc', ascending = False)[['ticker','acc']])['ticker'].unique().tolist()

In [1646]:
up_list = up_[up_['acc'] > 0.7].sort_values('acc', ascending = False)['ticker'].unique().tolist()

In [1647]:
down_list = down_[down_['acc'] > 0.7].sort_values('acc', ascending = False)['ticker'].unique().tolist()

In [1648]:
# pd.DataFrame({'tickers': all}).to_csv('prod_tickers_top.csv',index = False)
# pd.DataFrame({'tickers': up_list}).to_csv('prod_up_tickers_top.csv',index = False)
# pd.DataFrame({'tickers': down_list}).to_csv('prod_down_tickers_top.csv',index = False)

In [1609]:
pd.read_csv('prod_down_tickers.csv')['tickers'].apply(lambda x: x.split('.')[0]).unique().tolist()

['A',
 'BIIB',
 'DEI',
 'OLED',
 'CTLT',
 'NYCB',
 'MS',
 'MCHP',
 'RKT',
 'SQ',
 'LAZ',
 'OMC',
 'BKNG',
 'ENTG',
 'TDOC',
 'DCT',
 'V',
 'ABMD',
 'VRM',
 'GO',
 'WRK',
 'EVBG',
 'ICUI',
 'NFE',
 'PTON',
 'NVAX',
 'WBS',
 'CCL',
 'SMAR',
 'AMED',
 'XEC',
 'CRWD',
 'TPX',
 'ATUS',
 'QDEL',
 'ROKU',
 'IPGP',
 'FANG',
 'OSK',
 'PACW',
 'TER',
 'Z',
 'COTY',
 'ZM',
 'FHB',
 'JWN',
 'STNE',
 'ENPH',
 'PLUG',
 'CLF',
 'UA',
 'ZG',
 'PVH',
 'DE',
 'CZR',
 'GME',
 'ZNGA',
 'ADPT',
 'RUN',
 'EQT',
 'CPRI',
 'PINS',
 'PLAN',
 'LLY',
 'NCR',
 'NVDA',
 'DHI',
 'PENN',
 'CHGG',
 'CFG',
 'SPOT',
 'UTHR',
 'W',
 'AGO',
 'L',
 'MKSI',
 'HAL',
 'EVR',
 'GPS',
 'X',
 'OSH',
 'FTNT',
 'NTRA',
 'FOUR',
 'IAC',
 'TTD',
 'CC',
 'DVN',
 'MRO',
 'NUE',
 'PCTY',
 'NCNO',
 'VFC',
 'OLLI',
 'LRCX',
 'SAM',
 'LAD',
 'XLNX',
 'KSS',
 'SCCO',
 'BILL',
 'NVCR',
 'VIAC',
 'CREE',
 'NOV',
 'RNG',
 'SYF',
 'DNB',
 'FIVE',
 'HXL',
 'ALNY',
 'DISCK',
 'TWTR',
 'IONS',
 'SKX',
 'FSLY',
 'THO',
 'VST',
 'SLB',
 'BHF',
 'A

In [1597]:
down_[down_['acc'] > 0.4].sort_values('acc', ascending = False)['acc'].mean()

0.6749518701742284

In [1598]:
down_[down_['acc'] > 0.5].sort_values('acc', ascending = False)[['is_dir','down']].sum()

is_dir    2514
down      1749
dtype: int64

In [1639]:
down_[down_['acc'] > 0.7].sort_values('acc', ascending = False)[['is_dir','down']].sum()

is_dir    450
down      355
dtype: int64

In [1640]:
up_[up_['acc'] > 0.7].sort_values('acc', ascending = False)[['is_dir','up']].sum()

is_dir    404
up        320
dtype: int64

In [1641]:
355/450

0.7888888888888889

In [1642]:
320/404

0.7920792079207921

In [1558]:
1222/1815

0.6732782369146005

In [622]:
temp_ = temp_[temp_['is_vol'] == 1].groupby('ticker').sum().reset_index()

In [623]:
temp_['%_l2'] = temp_['label_2.5']/ temp_['is_vol']
temp_['%_v'] = temp_['vol']/ temp_['is_vol']

In [624]:
temp_.sort_values(['%_l2','is_vol'], ascending =  [False, False]).reset_index(drop = True)

,ticker,is_vol,label_2.5,vol,%_l2,%_v
0,JWN.csv,73,73,40,1.0,0.547945
1,GPS.csv,72,72,33,1.0,0.458333
2,RNG.csv,60,60,26,1.0,0.433333
3,COMM.csv,59,59,28,1.0,0.474576
4,ATUS.csv,56,56,30,1.0,0.535714
...,...,...,...,...,...,...
96,REG.csv,2,0,0,0.0,0.000000
97,MTG.csv,1,0,0,0.0,0.000000
98,ROST.csv,1,0,0,0.0,0.000000
99,SNDR.csv,1,0,0,0.0,0.000000


In [625]:
temp_.sort_values(['is_vol'], ascending =  [False])

,ticker,is_vol,label_2.5,vol,%_l2,%_v
51,JWN.csv,73,73,40,1.000000,0.547945
40,GPS.csv,72,72,33,1.000000,0.458333
79,ROKU.csv,71,69,35,0.971831,0.492958
61,NVCR.csv,67,66,32,0.985075,0.477612
78,RNG.csv,60,60,26,1.000000,0.433333
...,...,...,...,...,...,...
84,SNDR.csv,1,0,0,0.000000,0.000000
43,HLT.csv,1,1,1,1.000000,1.000000
80,ROST.csv,1,0,0,0.000000,0.000000
48,IT.csv,1,1,0,1.000000,0.000000


In [1662]:
temp_.sort_values(['%_l2','is_vol'], ascending =  [False, False]).reset_index(drop = True)

,ticker,is_vol,label_2.5,vol,%_l2,%_v
0,MSFT.csv,1,1,0,1.000000,0.000000
1,FB.csv,51,44,25,0.862745,0.490196
2,AAPL.csv,23,19,7,0.826087,0.304348
3,GOOG.csv,13,9,1,0.692308,0.076923
4,AMZN.csv,17,7,2,0.411765,0.117647


In [1596]:
temp_.sort_values(['is_vol'], ascending =  [False])

,ticker,is_vol,label_2.5,vol,%_l2,%_v
2,FB.csv,80,65,57,0.812500,0.712500
0,AAPL.csv,43,32,22,0.744186,0.511628
1,AMZN.csv,39,20,17,0.512821,0.435897
3,GOOG.csv,30,18,12,0.600000,0.400000
4,MSFT.csv,7,6,4,0.857143,0.571429


In [1597]:
temp_.sort_values(['%_l2','is_vol'], ascending =  [False, False]).reset_index(drop = True)

,ticker,is_vol,label_2.5,vol,%_l2,%_v
0,MSFT.csv,7,6,4,0.857143,0.571429
1,FB.csv,80,65,57,0.812500,0.712500
2,AAPL.csv,43,32,22,0.744186,0.511628
3,GOOG.csv,30,18,12,0.600000,0.400000
4,AMZN.csv,39,20,17,0.512821,0.435897


In [1685]:
temp_.sort_values(['%_l2','is_vol'], ascending =  [False, False]).reset_index(drop = True)[:12].sum()

ticker       CSCO.csvJNJ.csvMCD.csvPG.csvAME.csvFB.csvV.csv...
is_vol                                                     480
label_2.5                                                  357
vol                                                        189
%_l2                                                  9.933364
%_v                                                   4.152545
dtype: object

In [1686]:
357/480

0.74375

In [565]:
temp.sort_values(['%_l4','is_vol'], ascending =  [False, False])[49:99]

KeyError: '%_l4'

In [152]:
test_df['is_vol'] = o

# test_df['direction'] = o_d

In [96]:
test_df[(test_df['ticker'] == 'RBC.csv') & (test_df['is_vol'] == 1)].columns.tolist()

['12d_ch_v',
 '12d_vol',
 '1d_DI+14_ch_p',
 '1d_DI+14_ushift',
 '1d_DI+20_ch_p',
 '1d_DI+20_ushift',
 '1d_DI-14_ch_p',
 '1d_DI-14_ushift',
 '1d_DI-20_ch_p',
 '1d_DI-20_ushift',
 '1d_RSI14_ch_p',
 '1d_RSI14_ushift',
 '1d_adx14_ch_p',
 '1d_adx14_ushift',
 '1d_adx9_ch_p',
 '1d_adx9_ushift',
 '1d_ch_p',
 '1d_ch_v',
 '1d_ushift',
 '1d_vol',
 '1ushift',
 '1ushift_h',
 '1ushift_l',
 '1ushift_o',
 '1ushift_v',
 '21d_ch_v',
 '21d_vol',
 '2d_DI+14_ch_p',
 '2d_DI+14_ushift',
 '2d_DI+20_ch_p',
 '2d_DI+20_ushift',
 '2d_DI-14_ch_p',
 '2d_DI-14_ushift',
 '2d_DI-20_ch_p',
 '2d_DI-20_ushift',
 '2d_RSI14_ch_p',
 '2d_RSI14_ushift',
 '2d_adx14_ch_p',
 '2d_adx14_ushift',
 '2d_adx9_ch_p',
 '2d_adx9_ushift',
 '2d_ch_p',
 '2d_ushift',
 '3d_DI+14_ch_p',
 '3d_DI+14_ushift',
 '3d_DI+20_ch_p',
 '3d_DI+20_ushift',
 '3d_DI-14_ch_p',
 '3d_DI-14_ushift',
 '3d_DI-20_ch_p',
 '3d_DI-20_ushift',
 '3d_RSI14_ch_p',
 '3d_RSI14_ushift',
 '3d_adx14_ch_p',
 '3d_adx14_ushift',
 '3d_adx9_ch_p',
 '3d_adx9_ushift',
 '3d_ch_p',
 '3

In [532]:
test_df[(test_df['is_vol'] == 1)  & (test_df['direction'] == 1) & (test_df['future_ch_p']  > 0.02) ].shape

(62, 174)

In [533]:
test_df[(test_df['is_vol'] == 1)  & (test_df['direction'] == 1) & (test_df['max_ch'] > 0.02) ].shape

(97, 174)

In [534]:
test_df[(test_df['is_vol'] == 1) & (test_df['direction'] == 1)].shape

(131, 174)

In [535]:
test_df[(test_df['is_vol'] == 1)  & (test_df['direction'] == 0) & (test_df['future_ch_p'] < -0.02) ].shape

(15, 174)

In [536]:
test_df[(test_df['is_vol'] == 1)  & (test_df['direction'] == 0) & (test_df['min_ch'] < -0.02) ].shape

(25, 174)

In [537]:
test_df[(test_df['is_vol'] == 1) & (test_df['direction'] == 0)].shape

(30, 174)

In [490]:
test_df[(test_df['is_vol'] == 1) & (test_df['direction'] == 1)]['future_ch_p'].sum()

2.013336103509154

In [542]:
temp = test_df[(test_df['is_vol'] == 1) & (test_df['direction'] == 1) ]
temp['future_ch_p'] = temp['future_ch_p'] * 100

In [543]:
temp['future_ch_p'] = temp['future_ch_p'].apply(lambda x: 11.2 if x >= 11.2 else x)
temp['future_ch_p'] = temp['future_ch_p'].apply(lambda x: -8 if x <= -8 else x)

def ch(x,y):
  if x >= 0.112:
    return 11.2
  else:
    return y

def ch_(x,y):
  if x <= -0.08:
    return -8
  else:
    return y

In [544]:
temp['future_ch_p'] = temp.apply(lambda x: ch(x.max_ch, x.future_ch_p), axis = 1)
temp['future_ch_p'] = temp.apply(lambda x: ch_(x.min_ch, x.future_ch_p), axis = 1)

In [545]:
temp['future_ch_p'] = temp['future_ch_p'] - 0.2
temp['future_ch_p'].sum()

170.28008331567722

In [546]:
test_df[(test_df['is_vol'] == 1) &  (test_df['direction'] == 0)].shape

(30, 174)

In [547]:
temp = test_df[(test_df['is_vol'] == 1) &  (test_df['direction'] == 0)]
temp['future_ch_p'] = temp['future_ch_p'] * 100
temp['future_ch_p'] = temp['future_ch_p'].apply(lambda x: 8 if x >= 8 else x)
temp['future_ch_p'] = temp['future_ch_p'].apply(lambda x: -11.2 if x <= -11.2 else x)

def ch(x,y):
  if x > 0.08:
    return 8
  else:
    return y

def ch_(x,y):
  if x <= -0.112:
    return -11.2
  else:
    return y

# temp['future_ch_p'] = temp.apply(lambda x: ch(x.max_ch, x.future_ch_p), axis = 1)
temp['future_ch_p'] = temp.apply(lambda x: ch_(x.min_ch, x.future_ch_p), axis = 1)
temp['future_ch_p'] = temp.apply(lambda x: ch(x.max_ch, x.future_ch_p), axis = 1)
temp['future_ch_p'] = temp['future_ch_p'] * -1
temp['future_ch_p'] = temp['future_ch_p'] - 0.2

In [548]:
temp['future_ch_p'].sum()

41.882581592957415

In [500]:
temp[temp['future_ch_p'] > 0].shape

(20, 174)

In [235]:
113/196

0.576530612244898

In [69]:
temp[['Date','future_ch_p','min_ch','max_ch','ticker','label_2d']]['future_ch_p'].value_counts()

-7.000000     24
 11.200000    10
-3.047074      1
-0.372040      1
 1.531394      1
              ..
 2.771730      1
 8.171960      1
-1.907302      1
-0.323316      1
-2.854523      1
Name: future_ch_p, Length: 151, dtype: int64

In [434]:
temp['future_ch_p'] = temp['future_ch_p'] - 0.2

In [388]:
temp['future_ch_p'].sum()

-660.6060299110659

In [72]:
temp[temp['future_ch_p'] > 0]['future_ch_p'].sum()

348.8157025908854

In [73]:
temp[temp['future_ch_p'] < 0]['future_ch_p'].sum()

-315.90722133490607

In [58]:
temp[temp['future_ch_p'] <= 0]['future_ch_p'].sum()

-1347.8401266859073

In [59]:
temp[temp['future_ch_p'] > 0]['future_ch_p'].sum()

1668.2113166424288

In [ ]:
(252)/ 447

In [ ]:
temp = temp.sort_values('Date')

In [229]:
temp[['Date','future_ch_p','min_ch','max_ch','ticker','label_2d']]['future_ch_p'].value_counts()

 7.000000    29
-7.000000    23
-2.246321     1
-2.443580     1
-6.994104     1
 6.493919     1
 2.251407     1
-3.316704     1
 1.739130     1
 0.881461     1
-1.049916     1
 2.771730     1
-1.608177     1
-0.321623     1
-5.142379     1
-1.407344     1
 0.955978     1
-4.446104     1
-3.094883     1
 0.424925     1
-1.434810     1
 2.813858     1
-1.925867     1
-0.736936     1
-3.256925     1
-4.519628     1
 0.054215     1
-3.944125     1
 1.240444     1
-0.654952     1
 1.700999     1
-2.974339     1
 2.285783     1
-5.238955     1
 4.036842     1
 1.867302     1
 1.321586     1
 1.769351     1
-5.679245     1
-3.381818     1
-2.349558     1
-2.291536     1
 0.976915     1
Name: future_ch_p, dtype: int64

In [ ]:
temp[['Date','future_ch_p','min_ch','max_ch','ticker','label_2d']]['label_2d'].value_counts(normalize = True)

In [ ]:
ch[(ch['Volatality_Probability'] > 0.75) & (ch['Up_Direction_Probability'] > 0.7)]

In [ ]:
ch = pd.read_csv('2021-09-16-3d_Up_predicted.csv')

In [ ]:
ch[(ch['Volatality_Probability'] > 0.75) & (ch['Up_Direction_Probability'] > 0.7)]

In [ ]:
temp[['Date','future_ch_p','min_ch','max_ch','ticker']]

In [ ]:
label_df = new_df["label_4"][:train_len]

test_label_df = new_df["label_4"][train_len:]
label_np = label_df.to_numpy().astype(np.float32)
test_label_np = test_label_df.to_numpy().astype(np.float32)

In [ ]:
lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(26,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = BatchNormalization()(x)
x = Bidirectional(LSTM(60,return_sequences = True, activation = 'relu'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)

# x = Dense(30, activation='relu')(x)
x = Dropout(0.2)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = BatchNormalization()(y)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)

# y = Dense(30, activation='relu')(y)
y = Dropout(0.2)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(26,))(technicals)
z = BatchNormalization()(z)
z = Dense(30, activation='relu')(z)
z = Dropout(0.4)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
# combined = Dropout(0.1)(combined)
combined = Dense(40, activation='relu')(x.output)
combined = BatchNormalization()(combined)
combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])
# custom_loss
# compile the model
model.compile(optimizer = tf.optimizers.Nadam(learning_rate = 0.0001), loss = loss_fn, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 10, batch_size=256, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 10, batch_size=256, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
model.load_weights('3d_5p_weights.h5')

In [ ]:
# # fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 5, batch_size=1024, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
# # fit the model
# model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 5, batch_size=1024, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

# model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
lstm_seq_size = 40


technicals = Input(shape=(26,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the third branch opreates on the third input - technicals
z = Dense(100, activation='relu', input_shape=(26,))(technicals)
z = Dense(50, activation='relu')(z)
z = Dense(50, activation='relu')(z)
# z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
# combined = concatenate([x.output, z.output])
combined = Dropout(0.4)(z.output)
combined = Dense(30, activation='relu')(combined)
combined = Dense(30, activation='relu')(combined)
combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs= z.input, outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = loss_fn, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit( X_train_np , label_np, epochs= 40, batch_size=512, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)
model.evaluate( X_test_np , test_label_np)

p = model.predict( X_test_np)

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
pr = pd.DataFrame({'val':p[:,1].tolist()})


In [ ]:

pr['pred'] = pr['val'].apply(lambda x: 1 if x >= 0.55 else 0)

In [ ]:
p = model.predict( X_test_np)
pr['pred'] = pr['val'].apply(lambda x: 1 if x >= 0.65 else 0)
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
preds_class_ = model_.predict(eval_data)
# Get predicted probabilities for each class
preds_proba_ = model_.predict_proba(eval_data)
# Get predicted RawFormulaVal
preds_raw_ = model_.predict(eval_data, prediction_type='RawFormulaVal')

pr_ = pd.DataFrame({'val':preds_proba_[:,1].tolist()})
pr_['pred'] = pr_['val'].apply(lambda x: 1 if x >= 0.6 else 0)
o_ = []
for i in pr_['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o_.append(i)

In [ ]:
test_df['label_3d'].value_counts(normalize = True)

In [ ]:
test_df['direction'] = o_
test_df['is_vol'] = o

In [ ]:
temp = test_df[(test_df['is_vol'] == 1) ][['direction','label_3d','label_3']]

In [ ]:
len(test_df[(test_df['direction'] == 1) & (test_df['is_vol'] == 1) ][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
len(test_df[(test_df['direction'] == 1)  & (test_df['is_vol'] == 1) & (test_df['label_3d'] == 1)][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
15/24

In [ ]:
test_df[(test_df['direction'] == 1)  & (test_df['is_vol'] == 1) & (test_df['label_2d'] == 1)][['7d_ch_p','label_3','label_2d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
len(test_df[(test_df['direction'] == 1)  & (test_df['is_vol'] == 1)])

In [ ]:
test_df[(test_df['direction'] == 0)  & (test_df['is_vol'] == 1)  ][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date','direction']].sort_values(by = 'Date')

In [ ]:
len(temp[(temp['direction'] == 1) & (temp['label_3d'] == 1) & (temp['label_3'] == 1)])

In [ ]:
len(temp[(temp['direction'] == 0) & (temp['label_3d'] == 2)])

In [ ]:
len(test_df[(test_df['direction'] == 1)  & (test_df['is_vol'] == 1) ][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
len(test_df[(test_df['direction'] == 0)  & (test_df['is_vol'] == 1) ][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
len(test_df[(test_df['direction'] == 1)  & (test_df['is_vol'] == 1) & (test_df['label_3d'] == 1)][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
len(test_df[(test_df['direction'] == 0)  & (test_df['is_vol'] == 1) & (test_df['label_3d'] == 2)][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date'))

In [ ]:
test_df[(test_df['direction'] == 0)  & (test_df['is_vol'] == 1) ][['7d_ch_p','label_3','label_3d','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
len(temp[(temp['direction'] == 1)])

In [ ]:
len(temp[(temp['direction'] == 1) & (temp['label_3d'] == 2) & (temp['label_3'] == 1)])

In [ ]:
len(temp[(temp['direction'] == 0) & (temp['label_3d'] == 2) & (temp['label_3'] == 1)])

In [ ]:
len(test_df[(test_df['direction'] == 0) & (test_df['label_3d'] == 2)])

In [ ]:
len(test_df[(test_df['direction'] == 1) & (test_df['label_3d'] == 1)])

In [ ]:
len(test_df[(test_df['direction'] == 0)])

In [ ]:
o = []
for i in pr['pred'].tolist():

    
    o.append(i)
    
print(confusion_matrix(test_df['label_3'].tolist(), o))

print(classification_report(test_df['label_3'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_2.5'].tolist(), o))

print(classification_report(test_df['label_2.5'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1.5'].tolist(), o))

print(classification_report(test_df['label_1.5'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1.4'].tolist(), o))

print(classification_report(test_df['label_1.4'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1.35'].tolist(), o))

print(classification_report(test_df['label_1.35'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():

    
    o.append(i)
    
print(confusion_matrix(test_df['label_1.25'].tolist(), o))

print(classification_report(test_df['label_1.25'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_0.5'].tolist(), o))

print(classification_report(test_df['label_0.5'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1.25'].tolist(), o))

print(classification_report(test_df['label_1.25'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1'].tolist(), o))

print(classification_report(test_df['label_1'].tolist(), o))

In [ ]:
test_df['label_2d'].value_counts(normalize = True)

In [ ]:
test_df['label_2'].value_counts(normalize = True)

In [ ]:
test_df['label_0.5'].value_counts(normalize = True)

In [ ]:
test_df['label_0.7'].value_counts(normalize = True)

In [ ]:
test_df['label_0.6'].value_counts(normalize = True)

In [ ]:
test_df['label_1.5'].value_counts(normalize = True)

In [ ]:
test_df['label_1.25'].value_counts(normalize = True)

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_2'].tolist(), o))

print(classification_report(test_df['label_2'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1'].tolist(), o))

print(classification_report(test_df['label_1'].tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_df['label_1'].tolist(), o))

print(classification_report(test_df['label_1'].tolist(), o))

In [ ]:
from catboost import CatBoostClassifier
# Initialize data
# cat_features = [0, 1]
train_data =  X_train_np
train_labels = label_np
eval_data = X_test_np

# Initialize CatBoostClassifier
model = CatBoostClassifier(depth = 8)
# Fit model
model.fit(train_data, train_labels)
# Get predicted classe
preds_class = model.predict(eval_data)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_data)
# Get predicted RawFormulaVal
preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

pr = pd.DataFrame({'val':preds_proba[:,1].tolist()})

In [ ]:
pr = pd.DataFrame({'val':preds_proba[:,1].tolist()})

In [ ]:
pr = pd.DataFrame({'val1':preds_proba[:,1].tolist(),'val2':preds_proba[:,2].tolist()})

In [ ]:
pr = pd.DataFrame({'val1':p[:,1].tolist(),'val2':p[:,2].tolist()})

In [ ]:
pr = pr.reset_index()

In [ ]:
o = []
for i in pr['index'].tolist():
    if (pr.iloc[i]['val1'] > 0.4) & (pr.iloc[i]['val1'] > pr.iloc[i]['val2']):
        o.append(1)
    elif (pr.iloc[i]['val2'] > 0.4) & (pr.iloc[i]['val2'] > pr.iloc[i]['val1']):
        o.append(2)
    else:
        o.append(0)
    
#     o.append(i)

In [ ]:
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
pr['pred'] = pr['val'].apply(lambda x: 1 if x >= 0.85 else 0)

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
o = []
for i in pr['pred'].tolist():
#     if i[1] > 0.7:
#         o.append(1)
#     else:
#         o.append(o)
    
    o.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(14,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(14,))(technicals)
z = Dense(30, activation='relu')(z)
# z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, z.output])
combined = Dropout(0.1)(z.output)
# combined = Dense(40, activation='relu')(combined)
# combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.005), loss = loss_fn, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 40, batch_size=256, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)
model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
model.save('3d_6p_weights.h5')

In [ ]:
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 6, batch_size=256, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)
model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
test_df['predicted'] = o

In [ ]:
def che(x):
    min_ = test_df.iloc[x]['min_ch']
    max_ = test_df.iloc[x]['max_ch']
    if (max_ >= 0.012) :
        return 1
    elif (min_ <= -0.012):
        return 1
    else:
        return 0
    


In [ ]:
# test_df = test_df.drop(columns = ['index','level_0'])

In [ ]:
test_df = test_df.reset_index(drop = True).reset_index()
test_df['label_1.2'] = test_df['index'].apply(che)

In [ ]:
test_df[-1:]['Date']

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-03-01')  & (test_df['Date'] <= '2021-09-15')][['7d_ch_p','label_2','label_1','label_0.5','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-05-01')  & (test_df['Date'] <= '2021-09-15')][['7d_ch_p','label_2','label_1','label_0.5','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-01-01')  & (test_df['Date'] <= '2021-09-15')][['7d_ch_p','label_1.25','label_1.2','label_1','label_0.5','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-01-01')  & (test_df['Date'] <= '2021-09-15')][['7d_ch_p','label_0.7','label_0.9','label_1','label_0.5','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-01-01')  & (test_df['Date'] <= '2021-08-15')][['7d_ch_p','label_6','label_5','label_4','label_3','label_2','label_1','label_1.8','ticker','min_ch','max_ch','future_ch_p','Date']].sort_values(by = 'Date')

In [ ]:
test_df[ (test_df['predicted'] == 1)   & (test_df['Date'] >= '2021-08-01')  & (test_df['Date'] <= '2021-09-01')][['7d_ch_p','temp_la','label','ticker','min_ch','max_ch','future_ch_p','Date','Adj_open','Adj_high']].sort_values(by = 'Date')

In [ ]:
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 5, batch_size=256, validation_split = 0.3, callbacks=[earlyStopping, mcp_save], verbose=1)
model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
pd.options.mode.chained_assignment = None

tick = []
av = ['AGC.csv',
 'LMAO.csv',
 'OG.csv',
 'U.csv',
 'ACKIT.csv',
 'CCIV.csv',
 'RSI.csv',
 '.DS_Store',
 'UWMC.csv',
 'CRSR.csv',
 'ACND.csv',
 'BREZ.csv',
 'ACEVW.csv',
 'BDTX.csv',
 'PSFE.csv',
 'RBLX.csv',
 'ASO.csv',
 'RKT.csv',
 'PLTR.csv',
 'ANAC.csv']
# av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
for i in os.listdir('training_data'):
    if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av) :
        tick.append(i)

snp_train = pd.read_csv('training_data/VOO.csv')
snp_test = pd.read_csv('testing_data/VOO.csv')

snp_train = snp_train[['Date','future_ch_p','Change_Seq']]
snp_test = snp_test[['Date','future_ch_p','Change_Seq']]

snp_train.columns = ['Date','snp_future_ch_p','snp_Change_Seq']
snp_test.columns = ['Date','snp_future_ch_p','snp_Change_Seq']

df_list = []

for i in tick[:60]:
    df_list.append(pd.read_csv('training_data/' + i))
    

df_test = []

for i in tick[:60]:
    if i != '.DS_Store':
        df_test.append(pd.read_csv('testing_data/' + i))
        
new_df = pd.concat(df_list, ignore_index=True, sort=True)
    
test_df = pd.concat(df_test, ignore_index=True, sort=True)

new_df = new_df.merge(snp_train, on = 'Date')
test_df = test_df.merge(snp_test, on = 'Date')

new_df["latest_change"] = new_df["future_ch_p"] - new_df["snp_future_ch_p"]
test_df["latest_change"] = test_df["future_ch_p"] - test_df["snp_future_ch_p"]

new_df["3l"]=new_df["latest_change"].apply(add3LIndicator_3D)
new_df["label"] = new_df["3l"].apply(addLabel_3D)

test_df["3l"]=test_df["latest_change"].apply(add3LIndicator_3D)
test_df["label"] = test_df["3l"].apply(addLabel_3D)

new_df = new_df.sample(frac=1).reset_index(drop=True)
# new_df = new_df.sample(frac=0.30).reset_index(drop = True)
train_len = len(new_df)

new_df = new_df.append(test_df)

price_df = new_df["Price_Seq"]
snp_price_df = new_df["snp_Change_Seq"]
volume_df = new_df["Volume_Seq"]
change_df = new_df['Change_Seq']

price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []

print('completed------1 \n')

for i in range(price_df.shape[0]):
    price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
    snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
    volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))
    pattern_lstm_.append(convToNumpyArr(change_df.iloc[i],4))
    
price_lstm_np = np.array(price_lstm_)
snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_)
pattern_lstm_np = np.array(pattern_lstm_)

pattern_lstm_np = pattern_lstm_np - snp_price_lstm_np

test_price_lstm_np = price_lstm_np[train_len:]
price_lstm_np = price_lstm_np[:train_len]

test_volume_lstm_np = volume_lstm_np[train_len:]
volume_lstm_np = volume_lstm_np[:train_len]

test_pattern_lstm_np = pattern_lstm_np[train_len:]
pattern_lstm_np = pattern_lstm_np[:train_len]

label_df = new_df["label"][:train_len]
test_label_df = new_df["label"][train_len:]
label_np = label_df.to_numpy()
test_label_np = test_label_df.to_numpy()

print('completed------2 \n')

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
mcp_save = ModelCheckpoint('weigts.h5', save_best_only=True, monitor='val_accuracy', mode='min')

X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

X_train_df = pd.concat([X_train1_df[:train_len], X_train2_df[:train_len], X_train3_df[:train_len]], axis=1)
X_test_df = pd.concat([X_train1_df[train_len:], X_train2_df[train_len:], X_train3_df[train_len:]], axis=1)

X_train_np = np.array(X_train_df)
X_test_np = np.array(X_test_df)

# define two sets of inputs

lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(10,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Dense(40, activation='relu')(combined)
#combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.003), loss = custom_loss, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 10, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(10,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
z = Dense(30, activation='relu')(z)
# z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(z.output)
#combined = Dense(40, activation='relu')(combined)
#combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(3, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = loss_fn, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 20, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:

lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(10,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Dense(40, activation='relu')(combined)
#combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(3, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.003), loss = custom_loss, metrics=[f1_score, 'accuracy', precision_m])


print(model.summary())


# fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 10, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

In [ ]:
model.save_weights('weights_3d_snp.h5')

In [ ]:
# voo_train = pd.read_csv('training_data/VOO.csv')
# voo_test = pd.read_csv('testing_data/VOO.csv')

# voo_train = voo_train[['Date','future_ch_p','Change_Seq']]
# voo_train.columns = ['Date','VOO_change','VOO_Change_Seq']

# voo_test = voo_test[['Date','future_ch_p','Change_Seq']]
# voo_test.columns = ['Date','VOO_change','VOO_Change_Seq']

# new_df = new_df.merge(voo_train, on = 'Date')
# test_df = test_df.merge(voo_test, on = 'Date')

# new_df['diff'] = new_df['future_ch_p'] - new_df['VOO_change']
# test_df['diff'] = test_df['future_ch_p'] - test_df['VOO_change']

# new_df["direction"]=new_df["diff"].apply(add3LIndicator)
# new_df['label'] = new_df["direction"].apply(addLabel)

# test_df["direction"]=test_df["diff"].apply(add3LIndicator)
# test_df['label'] = test_df["direction"].apply(addLabel)

# price_df = new_df["Price_Seq"]
# volume_df = new_df["Volume_Seq"]
# price_lstm_ = []
# volume_lstm_ = []

# for i in range(price_df.shape[0]):
#     price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
#     volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))

# price_lstm_np = price_lstm_np - snp_price_lstm_np

# price_lstm_np = np.array(price_lstm_)
# volume_lstm_np = np.array(volume_lstm_)

# test_price_lstm_np = np.array(test_price_lstm)
# test_volume_lstm_np = np.array(test_volume_lstm)

In [ ]:
pd.options.mode.chained_assignment = None

features = ['%_neg_ch', '%_neg_ch2', '%_neg_ch3', '%_neg_ch7', '%_neu_ch', '%_neu_ch2',
       '%_neu_ch3', '%_neu_ch7', '%_ovr_ch', '%_ovr_ch2', '%_ovr_ch3','%_ovr_ch7', '%_pos_ch',
       '%_pos_ch2', '%_pos_ch3','%_pos_ch7',  'Negative', 'Neutral', 'Positive',  'change_negative',
       'change_negative_2', 'change_negative_3', 'change_neutral',
       'change_neutral_2', 'change_neutral_3', 'change_overall',
       'change_overall_2', 'change_overall_3', 'change_positive',
       'change_positive_2', 'change_positive_3','change_negative_7', 'change_neutral_7',
       'change_positive_7', 'change_overall_7']

tick = []
av = ['AGC.csv',
 'LMAO.csv',
 'OG.csv',
 'U.csv',
 'ACKIT.csv',
 'CCIV.csv',
 'RSI.csv',
 '.DS_Store',
 'UWMC.csv',
 'CRSR.csv',
 'AIO.csv',
 'ACND.csv',
 'BREZ.csv',
 'ACEVW.csv',
 'BDTX.csv',
 'PSFE.csv',
 'RBLX.csv',
 'ASO.csv',
 'RKT.csv',
 'PLTR.csv',
 'ANAC.csv',
 'AFAQ.csv']
# av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
for i in os.listdir('reddit_train'):
    if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
        tick.append(i)
        
# df_list = []

# for i in tick:
#     df_list.append(pd.read_csv('reddit_train/' + i))
    

df_list = []

for i in tick[:100]:
    df_list.append(pd.read_csv('reddit_train/' + i))
    
new_df = pd.concat(df_list, ignore_index=True, sort=True)


df_test = []
for i in tick[100:]:
    df_test.append(pd.read_csv('reddit_train/' + i))
    
test_df = pd.concat(df_test, ignore_index=True, sort=True)
    

new_df= new_df.sort_values('Date').reset_index(drop = True)
test_df =test_df.sort_values('Date').reset_index(drop = True)
    
# test_df = pd.concat(df_test, ignore_index=True, sort=True)

# train_len = len(new_df)

new_df = new_df.append(test_df)

price_df = new_df["Price_Seq"]
# snp_price_df = new_df["VOO_Change_Seq"]
volume_df = new_df["Volume_Seq"]
change_df = new_df['Change_Seq']

price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []

for i in range(price_df.shape[0]):
    price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
#     snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
    volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))
    pattern_lstm_.append(convToNumpyArr(change_df.iloc[i],4))

price_df_ = test_df["Price_Seq"]
# snp_price_df = new_df["VOO_Change_Seq"]
volume_df_ = test_df["Volume_Seq"]
change_df_ = test_df['Change_Seq']

test_price_lstm_ = []
test_snp_price_lstm_ = []
test_volume_lstm_ = []
test_pattern_lstm_ = []

for i in range(price_df_.shape[0]):
    test_price_lstm_.append(convToNumpyArr(price_df_.iloc[i], 4))
#     snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
    test_volume_lstm_.append(convToNumpyArr(volume_df_.iloc[i], 2))
    test_pattern_lstm_.append(convToNumpyArr(change_df_.iloc[i],4))

print('completed------1 \n')

price_lstm_np = np.array(price_lstm_)
# snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_)
pattern_lstm_np = np.array(pattern_lstm_)

test_price_lstm_np = np.array(test_price_lstm_)
# snp_price_lstm_np = np.array(snp_price_lstm_)
test_volume_lstm_np = np.array(test_volume_lstm_)
test_pattern_lstm_np = np.array(test_pattern_lstm_)

print('completed------2 \n')

label_df = new_df["label"]
test_label_df = test_df["label"]
label_np = label_df.to_numpy()
test_label_np = test_label_df.to_numpy()

lstm_seq_size = 40

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
mcp_save = ModelCheckpoint('weigts.h5', save_best_only=True, monitor='val_accuracy', mode='min')

X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

X_test1_df = (test_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_test2_df = test_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_test3_df = test_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

X_train_df = pd.concat([X_train1_df, X_train2_df, X_train3_df], axis=1)
X_test_df = pd.concat([X_test1_df, X_test2_df, X_test3_df], axis=1)

X_train_np = np.array(X_train_df)
X_test_np = np.array(X_test_df)

print('completed------3 \n')

# reddit features

X_train_reddit = new_df[features]
X_test_reddit = test_df[features]

X_train_reddit = np.array(X_train_reddit)
X_test_reddit = np.array(X_test_reddit)
# define two sets of inputs

price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(10,))

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)

# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)

# the third branch opreates on the third input - technicals
z = Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
z = Dense(50, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)

#the 4th branch
# the second branch opreates on the second input - volume
p = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(pattern)
p = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(p)
p = Dropout(0.1)(p)
p = Model(inputs=pattern, outputs=p)

print('completed------4 \n')

# combine the output of the two branches
# combined = concatenate([  y.output,z.output, p.output])
# combined_ = Dropout(0.1)(combined)
# combined_ = Dense(40, activation='relu')(combined_)
# combined_ = Dropout(0.1)(combined_)
# #combined = Multiply()([x.output, y.output])
# # apply a FC layer and then a regression prediction on the
# # combined outputs
# out = Dense(2)(combined_)
# our model will accept the inputs of the two branches and
# then output a single value
# model = Model(inputs=[  y.input, z.input, p.input], outputs=out)


#reddit branch
reddit = Input(shape=(35,))
r = Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(reddit)
r = Dense(50, activation='relu')(r)
r = Dropout(0.1)(r)
r = Model(inputs=reddit, outputs=r)

print('completed------5 \n')

# x_ = encoder(inputs=[  y.input, z.input, p.input])
x_ = encoder(inputs=[  y.input, z.input, p.input], training=False)

combined = concatenate([  x_ , r.output])
combined_ = Dropout(0.1)(combined)
combined_ = Dense(40, activation='relu')(combined_)
combined_ = Dropout(0.1)(combined_)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2)(combined_)

#encoder - used for our dimention reduction
# encoder = Model(inputs=[ x.input, y.input,z.input], outputs=combined)
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print('completed------6 \n')

# Convert features of shape `base_model.output_shape[1:]` to vectors
# A Dense classifier with a single unit (binary classification)

model = Model([  y.input, z.input, p.input, r.input], out)
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

print('completed------7 \n')
model.fit([volume_lstm_np, X_train_np, pattern_lstm_np,X_train_reddit], label_np, epochs=25, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

print('completed------8 \n')
p = model.predict([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

o = []
for i in p:
    o.append(i.argmax())
    

confusion_matrix(test_label_np.tolist(), o)



print(classification_report(test_label_np.tolist(), o))

In [ ]:
#reddit branch
reddit = Input(shape=(35,))
r = Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(reddit)
r = Dense(50, activation='relu')(r)
r = Dropout(0.1)(r)
r = Model(inputs=reddit, outputs=r)

print('completed------5 \n')

combined_ = Dropout(0.1)(r.output)
combined_ = Dense(40, activation='relu')(combined_)
combined_ = Dropout(0.1)(combined_)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2)(combined_)

#encoder - used for our dimention reduction
# encoder = Model(inputs=[ x.input, y.input,z.input], outputs=combined)
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print('completed------6 \n')

# Convert features of shape `base_model.output_shape[1:]` to vectors
# A Dense classifier with a single unit (binary classification)

model = Model([ r.input], out)
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

print('completed------7 \n')
model.fit([X_train_reddit], label_np, epochs=25, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([ X_test_reddit])

print('completed------8 \n')
p = model.predict([ X_test_reddit])

o = []
for i in p:
    o.append(i.argmax())
    

confusion_matrix(test_label_np.tolist(), o)



print(classification_report(test_label_np.tolist(), o))

In [ ]:
# pd.options.mode.chained_assignment = None

# features = ['%_neg_ch', '%_neg_ch2', '%_neg_ch3', '%_neg_ch7', '%_neu_ch', '%_neu_ch2',
#        '%_neu_ch3', '%_neu_ch7', '%_ovr_ch', '%_ovr_ch2', '%_ovr_ch3','%_ovr_ch7', '%_pos_ch',
#        '%_pos_ch2', '%_pos_ch3','%_pos_ch7',  'Negative', 'Neutral', 'Positive',  'change_negative',
#        'change_negative_2', 'change_negative_3', 'change_neutral',
#        'change_neutral_2', 'change_neutral_3', 'change_overall',
#        'change_overall_2', 'change_overall_3', 'change_positive',
#        'change_positive_2', 'change_positive_3','change_negative_7', 'change_neutral_7',
#        'change_positive_7', 'change_overall_7']

# tick = []
# av = ['AGC.csv',
#  'LMAO.csv',
#  'OG.csv',
#  'U.csv',
#  'ACKIT.csv',
#  'CCIV.csv',
#  'RSI.csv',
#  '.DS_Store',
#  'UWMC.csv',
#  'CRSR.csv',
#  'AIO.csv',
#  'ACND.csv',
#  'BREZ.csv',
#  'ACEVW.csv',
#  'BDTX.csv',
#  'PSFE.csv',
#  'RBLX.csv',
#  'ASO.csv',
#  'RKT.csv',
#  'PLTR.csv',
#  'ANAC.csv',
#  'AFAQ.csv']
# # av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
# for i in os.listdir('reddit_train'):
#     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
#         tick.append(i)
        
# # df_list = []

# # for i in tick:
# #     df_list.append(pd.read_csv('reddit_train/' + i))
    

# df_list = []

# for i in tick[:100]:
#     df_list.append(pd.read_csv('reddit_train/' + i))
    
# new_df = pd.concat(df_list, ignore_index=True, sort=True)


# df_test = []
# for i in tick[100:]:
#     df_test.append(pd.read_csv('reddit_train/' + i))
    
# test_df = pd.concat(df_test, ignore_index=True, sort=True)
    

# new_df= new_df.sort_values('Date').reset_index(drop = True)
# test_df =test_df.sort_values('Date').reset_index(drop = True)
    
# # test_df = pd.concat(df_test, ignore_index=True, sort=True)

# # train_len = len(new_df)

# new_df = new_df.append(test_df)

# price_df = new_df["Price_Seq"]
# # snp_price_df = new_df["VOO_Change_Seq"]
# volume_df = new_df["Volume_Seq"]
# change_df = new_df['Change_Seq']

# price_lstm_ = []
# snp_price_lstm_ = []
# volume_lstm_ = []
# pattern_lstm_ = []

# for i in range(price_df.shape[0]):
#     price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
# #     snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
#     volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))
#     pattern_lstm_.append(convToNumpyArr(change_df.iloc[i],4))

# price_df_ = test_df["Price_Seq"]
# # snp_price_df = new_df["VOO_Change_Seq"]
# volume_df_ = test_df["Volume_Seq"]
# change_df_ = test_df['Change_Seq']

# test_price_lstm_ = []
# test_snp_price_lstm_ = []
# test_volume_lstm_ = []
# test_pattern_lstm_ = []

# for i in range(price_df_.shape[0]):
#     test_price_lstm_.append(convToNumpyArr(price_df_.iloc[i], 4))
# #     snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
#     test_volume_lstm_.append(convToNumpyArr(volume_df_.iloc[i], 2))
#     test_pattern_lstm_.append(convToNumpyArr(change_df_.iloc[i],4))

# print('completed------1 \n')

# price_lstm_np = np.array(price_lstm_)
# # snp_price_lstm_np = np.array(snp_price_lstm_)
# volume_lstm_np = np.array(volume_lstm_)
# pattern_lstm_np = np.array(pattern_lstm_)

# test_price_lstm_np = np.array(test_price_lstm_)
# # snp_price_lstm_np = np.array(snp_price_lstm_)
# test_volume_lstm_np = np.array(test_volume_lstm_)
# test_pattern_lstm_np = np.array(test_pattern_lstm_)

# print('completed------2 \n')

# label_df = new_df["label"]
# test_label_df = test_df["label"]
# label_np = label_df.to_numpy()
# test_label_np = test_label_df.to_numpy()

# lstm_seq_size = 40

# earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('weigts.h5', save_best_only=True, monitor='val_accuracy', mode='min')

# X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
# X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
# X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

# X_test1_df = (test_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
# X_test2_df = test_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
# X_test3_df = test_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

# X_train_df = pd.concat([X_train1_df, X_train2_df, X_train3_df], axis=1)
# X_test_df = pd.concat([X_test1_df, X_test2_df, X_test3_df], axis=1)

# X_train_np = np.array(X_train_df)
# X_test_np = np.array(X_test_df)

# print('completed------3 \n')

# # reddit features

# X_train_reddit = new_df[features]
# X_test_reddit = test_df[features]

# X_train_reddit = np.array(X_train_reddit)
# X_test_reddit = np.array(X_test_reddit)
# # define two sets of inputs

# price = Input(shape=(lstm_seq_size,4))
# volume = Input(shape=(lstm_seq_size,2))
# pattern = Input(shape=(lstm_seq_size,4))
# technicals = Input(shape=(10,))

# # the first branch operates on the first input - price
# x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
# x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
# x = Dropout(0.1)(x)
# x = Model(inputs=price, outputs=x)

# # the second branch opreates on the second input - volume
# y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
# y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
# y = Dropout(0.1)(y)
# y = Model(inputs=volume, outputs=y)

# # the third branch opreates on the third input - technicals
# z = Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
# z = Dense(50, activation='relu')(z)
# z = Dropout(0.1)(z)
# z = Model(inputs=technicals, outputs=z)

# #the 4th branch
# # the second branch opreates on the second input - volume
# p = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(pattern)
# p = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(p)
# p = Dropout(0.1)(p)
# p = Model(inputs=pattern, outputs=p)

# print('completed------4 \n')

# # combine the output of the two branches
# # combined = concatenate([  y.output,z.output, p.output])
# # combined_ = Dropout(0.1)(combined)
# # combined_ = Dense(40, activation='relu')(combined_)
# # combined_ = Dropout(0.1)(combined_)
# # #combined = Multiply()([x.output, y.output])
# # # apply a FC layer and then a regression prediction on the
# # # combined outputs
# # out = Dense(2)(combined_)
# # our model will accept the inputs of the two branches and
# # then output a single value
# # model = Model(inputs=[  y.input, z.input, p.input], outputs=out)


# #reddit branch
# reddit = Input(shape=(35,))
# r = Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(reddit)
# r = Dense(50, activation='relu')(r)
# r = Dropout(0.1)(r)
# r = Model(inputs=reddit, outputs=r)

# print('completed------5 \n')

# x_ = encoder(inputs=[  y.input, z.input, p.input])

# combined = concatenate([  x_ , r.output])
# combined_ = Dropout(0.1)(combined)
# combined_ = Dense(40, activation='relu')(combined_)
# combined_ = Dropout(0.1)(combined_)
# #combined = Multiply()([x.output, y.output])
# # apply a FC layer and then a regression prediction on the
# # combined outputs
# out = Dense(2)(combined_)

# #encoder - used for our dimention reduction
# # encoder = Model(inputs=[ x.input, y.input,z.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# print('completed------6 \n')

# # Convert features of shape `base_model.output_shape[1:]` to vectors
# # A Dense classifier with a single unit (binary classification)

# model = Model([  y.input, z.input, p.input, r.input], out)
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# print('completed------7 \n')
# model.fit([volume_lstm_np, X_train_np, pattern_lstm_np,X_train_reddit], label_np, epochs=5, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

# model.evaluate([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

# print('completed------8 \n')
# p = model.predict([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

# o = []
# for i in p:
#     o.append(i.argmax())
    
# from sklearn.metrics import confusion_matrix

# confusion_matrix(test_label_np.tolist(), o)

In [ ]:
model.evaluate([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

print('completed------8 \n')
p = model.predict([ test_volume_lstm_np, X_test_np,test_pattern_lstm_np, X_test_reddit])

o = []
for i in p:
    o.append(i.argmax())
    
from sklearn.metrics import confusion_matrix

confusion_matrix(test_label_np.tolist(), o)

In [ ]:
# inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = encoder(inputs=[  y.input, z.input, p.input], training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# A Dense classifier with a single unit (binary classification)
outputs = Dense(2)(x)
model = Model([  y.input, z.input, p.input], outputs)

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])
model.fit([volume_lstm_np, X_train_np, pattern_lstm_np], label_np, epochs=5, batch_size=128, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label_np.tolist(), o))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label_np.tolist(), o))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_label_np.tolist(), o)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label_np.tolist(), o))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label_np.tolist(), o))

In [ ]:
c = 0
pre = []
for i in p:
    if c % 2 != 0:
        pre.append(i.argmax())
    c +=1
        

In [ ]:
for i in range(len(tick)):
    print(tick[i] + '-----------' + str(pre[i]))

In [ ]:
# define the model
model = Sequential()
model.add(Conv2D(64, 5, input_shape = (lstm_seq_size,6), activation = 'relu'))
model.add(Flatten())
model.add(Dense(3))

In [ ]:
# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.1), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit the model
model.fit(price_lstm_np, label_np, epochs=50, batch_size=64, verbose=1)
print(model.summary())

In [ ]:
#Model an LSTM

In [ ]:
#from keras import optimizers
#define the model
model = Sequential()
model.add(Bidirectional(LSTM(50,input_shape = (lstm_seq_size,6),return_sequences = False, activation = 'relu')))
model.add(Dropout(.2))
model.add(Dense(50, activation = 'relu'))
model.add(Dropout(.2))
model.add(Dense(3))

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn,metrics=['accuracy'])

In [ ]:
# fit the model
model.fit(price_lstm_np, label_np, epochs=100, batch_size=64, verbose=1)
print(model.summary())

In [ ]:
# define two sets of inputs
inputA = Input(shape=(32,))
inputB = Input(shape=(128,))
# the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)
x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(4, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
kukesh

In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

In [ ]:
#imp


lstm_seq_size = 40


price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(26,))

print('completed------3 \n')

# the first branch operates on the first input - price
# define two sets of inputs

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = BatchNormalization()(x)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)

# x = Dense(30, activation='relu')(x)
x = Dropout(0.2)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = BatchNormalization()(y)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)

# y = Dense(30, activation='relu')(y)
y = Dropout(0.2)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(26,))(technicals)
z = BatchNormalization()(z)
z = Dense(30, activation='relu')(z)
z = Dropout(0.4)(z)
z = Model(inputs=technicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
# combined = Dropout(0.1)(combined)
combined = Dense(40, activation='relu')(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)


print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])
# custom_loss
# compile the model
model.compile(optimizer = tf.optimizers.Nadam(learning_rate = 0.001), loss = loss_fn, metrics=[f1_score, 'accuracy', precision_m])


# print(model.summary())


# # fit the model
model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 5, batch_size=512, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)

model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np ], test_label_np)

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))